給定等長時間段：$[0, t_1)$, $[t_1, t_2)$, $[t_2, t_3)$, $[t_3, t_4)$, $[t_4, t_5)$, 給定$t_1 = 1, t_2 = 2, t_3 = 3, t_4 = 4, t_5 = 5$

Covariates: $Z_1 \sim N(0, 1)$, $Z_2\sim Ber(p=0.5)$

預期看到資料型態：

|ID|$N_{11}$|$N_{12}$|$N_{13}$|$N_{14}$|$N_{15}$|$N_{21}$|$N_{22}$|$N_{23}$|$N_{24}$|$N_{25}$|$Z_1$|$Z_2$|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|$\vdots$|$\vdots$|$\vdots$|$\vdots$|$\vdots$|$\vdots$|$\vdots$|$\vdots$|$\vdots$|$\vdots$|$\vdots$|$\vdots$|$\vdots$|

+ $N_{jk}$為第$j$種姿勢在第$k$段時間觀察到的次數。(本case中，$j=1,2$, $k=1, 2, 3, 4, 5$)
+ $Z_i$為covariates

---

兩種random effect: $\theta_1,\theta_2$來自Sarmanov distribution
$$
h(\theta_1, \theta_2) = h_1(\theta_1)h_2(\theta_2)(1 + \omega \phi_1(\theta_1)\phi_2(\theta_2))
$$

$$
\phi_j(\theta_j) = e^{-\theta_j} - L_{\theta_j}(1) = e^{-\theta_j} - (\frac{\alpha_j}{\alpha_j + 1})^{\alpha_j}, \,\,j = 1, 2
$$

---

模擬：

+ 觀察時間：1, 2, 3, 4, 5 $\rightarrow$ 共有 $[0, 1)$, $[1, 2)$, $[2, 3)$, $[3, 4)$, $[4, 5)$ 共五段時間。
+ True value: $\alpha_1 = 3$, $\alpha_2 = 2$, $\omega = 0.5$, $\beta_{10} = 0.5$, $\beta_{11}=0.5$, $\beta_{12}=-0.5$, $\beta_{20}=0.5$, $\beta_{21}=-0.5$, $\beta_{22}=0.5$
+ 第一種姿勢的$\Lambda$, $d\Lambda_{1k} = \int^{t2}_{t1} c_1 dt = c_1 \times (t_2 - t_1)$, $k=1, 2, 3, 4, 5$, $c_1=0.05$
+ 第二種姿勢的$\Lambda$, $d\Lambda_{2k} = \int^{t2}_{t1} c_2t dt = \frac{c_2}{2}(t^2_2 - t^2_1)$, $k=1, 2, 3, 4, 5$, $c_2=0.05$

---

待估計：
$$
\alpha_1, \alpha_2, \omega, \tilde{\beta}_1, \tilde{\beta}_2, \Lambda_1(d\Lambda_{11}, d\Lambda_{12}, \cdots), \Lambda_2(d\Lambda_{21}, d\Lambda_{22}, \cdots)
$$

---

In [2]:
generate.data = function(N, alpha, beta1, beta2, omega){
    # N: number of data
    # alpha: a vector of alpha to generate random effect theta1 and theta2
    # beta1, beta2: a vector of beta coefficients
    # omega: to generate random effect
    # generate two sets of random variable from uniform(0, 1)
    
    alpha1 = alpha[1]
    alpha2 = alpha[2]
    beta10 = beta1[1]
    beta11 = beta1[2]
    beta12 = beta1[3]
    beta20 = beta2[1]
    beta21 = beta2[2]
    beta22 = beta2[3]

    # Generate theta1 and theta2
    t1 = c()
    t2 = c()
    while (length(t1) < N){
        u1 = runif(1, 0, 1)
        z = runif(1, 0, 1)
        theta1 = qgamma(u1, alpha1, rate = alpha1)
        C = function(u2){
            f1 = exp(-1 * theta1) - (alpha1 / (alpha1 + 1))^alpha1
            f2 = (alpha2 / (alpha2 + 1))^alpha2 * (pgamma(qgamma(u2, alpha2, rate = alpha2), alpha2, rate = (alpha2 + 1)) - u2)
            return (u2 + omega * f1 * f2 - z)
        }
        r = unlist(unname(uniroot(C, lower = 0, upper = 1)[1]))
        theta2 = qgamma(r, alpha2, rate = alpha2)       
        if (theta1 != Inf && theta2 != Inf){
            t1 = c(t1, theta1)
            t2 = c(t2, theta2)
        }else{
            next
        }
    }

    # Generate covariate
    z1 = rnorm(N)
    z2 = sample(c(0, 1), N, p = c(.5, .5), replace = T)

    # Calculate the Lambda_i
    t = c(0, 1, 2, 3, 4, 5)
    lambda1_vec = c()
    lambda2_vec = c()
    for (i in 1:(length(t)-1)){
        v1 = 0.05 * (t[i+1] - t[i])
        v2 = integrate(f = function(x) 0.05 * x, lower = t[i], upper = t[i+1])$val
        lambda1_vec = c(lambda1_vec, v1)
        lambda2_vec = c(lambda2_vec, v2)
    }
    
    ind = length(lambda1_vec)
    n1_5 = c()
    n2_5 = c()    
    for (i in 1:ind){
        lambda1 = t1 * lambda1_vec[i] * exp(beta10 + beta11 * z1 + beta12 * z2)
        n1 = rpois(N, lambda = lambda1)
        n1_5 = cbind(n1_5, n1) 
    }

    for (i in 1:ind){
        lambda2 = t2 * lambda2_vec[i] * exp(beta20 + beta21 * z1 + beta22 * z2)
        n2 = rpois(N, lambda = lambda2)
        n2_5 = cbind(n2_5, n2)
    }

    data = cbind(n1_5, n2_5, z1, z2)
    colnames(data) = c("N11", "N12", "N13", "N14", "N15", "N21", "N22", "N23", "N24", "N25", "Z1", "Z2")
    return (data)
}

In [3]:
dt = generate.data(N = 300, alpha = c(3, 2), beta1 = c(0.5, 0.5, -0.5), beta2 = c(0.5, -0.5, 0.5), omega = 0.5)
print(head(dt, 30))

      N11 N12 N13 N14 N15 N21 N22 N23 N24 N25           Z1 Z2
 [1,]   0   0   0   0   0   0   1   0   1   2 -1.269423732  1
 [2,]   0   0   0   0   0   0   1   0   0   0  0.292793265  0
 [3,]   0   0   0   0   0   0   0   0   0   0 -0.563419929  0
 [4,]   0   0   0   0   0   0   0   0   0   0 -0.639706668  0
 [5,]   0   0   1   0   0   0   0   0   1   0  0.670522365  0
 [6,]   1   0   0   0   0   0   0   0   0   0  2.241465489  1
 [7,]   0   0   0   0   0   0   0   0   1   0 -0.193926939  1
 [8,]   0   0   0   0   0   0   0   0   0   1 -1.130590559  1
 [9,]   0   1   0   0   0   0   0   0   1   0 -0.603653554  0
[10,]   0   0   0   0   0   0   0   0   0   0  0.007984467  0
[11,]   0   0   0   1   0   0   0   0   0   0  0.440745362  0
[12,]   0   0   0   0   0   0   0   0   0   0  0.577018538  1
[13,]   0   0   0   0   0   0   0   0   0   0 -0.206042000  0
[14,]   0   0   0   1   0   0   1   1   1   0 -1.045457437  1
[15,]   0   0   0   0   0   0   1   0   0   0  0.012080400  1
[16,]   

---

\begin{align}
&P(\textbf{N}_i^{(1)} = \textbf{n}_i^{(1)}, \textbf{N}_i^{(2)} = \textbf{n}_i^{(2)}) = \underbrace{\binom{\sum^n_{j = 1}n_{ij}^{(1)}}{n_{i1}^{(1)}, \cdots, n_{in}^{(1)}}\prod^n_{j=1}p_{j1}^{n_{ij}^{(1)}}}_{multinomial} \times \underbrace{\binom{\sum^n_{j=1}n_{ij}^{(2)}}{n_{i1}^{(2)}, \cdots, n_{in}^{(2)}}\prod^n_{j=1}p_{j2}^{n_{ij}^{(2)}}}_{multinomial} \\
\times& \underbrace{\frac{\Gamma(\alpha_1 + \sum^n_{j=1}n_{ij}^{(1)})}{\Gamma(\alpha_1)(\sum^n_{j=1}n_{ij}^{(1)})!}\left(\frac{\alpha_1}{\alpha_1 + e^{\beta_1'Z }\sum^n_{j=1}\Lambda_{j1}}\right)^{\alpha_1} \left(\frac{e^{\beta_1'Z }\sum^n_{j=1}\Lambda_{j1}}{\alpha_1 + e^{\beta_1'Z }\sum^n_{j=1}\Lambda_{j1}}\right)^{\sum^n_{j=1}n_{ij}^{(1)}}}_{Negative Binomial}\\
\times& \underbrace{\frac{\Gamma(\alpha_2 + \sum^n_{j=1}n_{ij}^{(2)})}{\Gamma(\alpha_2)(\sum^n_{j=1}n_{ij}^{(2)})!}\left(\frac{\alpha_2}{\alpha_2 + e^{\beta_2'Z }\sum^n_{j=1}\Lambda_{j2}}\right)^{\alpha_2} \left(\frac{e^{\beta_2'Z }\sum^n_{j=1}\Lambda_{j2}}{\alpha_2 + e^{\beta_2'Z }\sum^n_{j=1}\Lambda_{j2}}\right)^{\sum^n_{j=1}n_{ij}^{(2)}}}_{Negative Binomial}\\
\times& \left[1 + \omega\left(\left(\frac{\alpha_1 + e^{\beta_1'Z}\sum^n_{j=1}\Lambda_{j1}}{\alpha_1 + e^{\beta_1'Z}\sum^n_{j=1}\Lambda_{j1}+1}\right)^{\alpha_1 + \sum^n_{j=1}n_{ij}^{(1)}} - \left(\frac{\alpha_1}{\alpha_1 + 1}\right)^{\alpha_1}\right) \left(\left(\frac{\alpha_2 + e^{\beta_2'Z}\sum^n_{j=1}\Lambda_{j2}}{\alpha_2 + e^{\beta_2'Z}\sum^n_{j=1}\Lambda_{j2}+1}\right)^{\alpha_2 + \sum^n_{j=1}n_{ij}^{(2)}} - \left(\frac{\alpha_2}{\alpha_2 + 1}\right)^{\alpha_2}\right) \right]
\end{align}

則loglikelihood為：

$$
L = \prod^m_{i=1} P(\textbf{N}_i^{(1)} = \textbf{n}_i^{(1)}, \textbf{N}_i^{(2)} = \textbf{n}_i^{(2)})
$$

先估計第一種姿勢的部份,其部份的 likelihood 命名為 L1：

其相關的部份有
$$
L1 = \prod^m_{i=1}\left[ \binom{\sum^n_{j = 1}n_{ij}^{(1)}}{n_{i1}^{(1)}, \cdots, n_{in}^{(1)}}\prod^n_{j=1}p_{j1}^{n_{ij}^{(1)}} \frac{\Gamma(\alpha_1 + \sum^n_{j=1}n_{ij}^{(1)})}{\Gamma(\alpha_1)(\sum^n_{j=1}n_{ij}^{(1)})!}\left(\frac{\alpha_1}{\alpha_1 + e^{\beta_1'Z }\sum^n_{j=1}\Lambda_{j1}}\right)^{\alpha_1} \left(\frac{e^{\beta_1'Z }\sum^n_{j=1}\Lambda_{j1}}{\alpha_1 + e^{\beta_1'Z }\sum^n_{j=1}\Lambda_{j1}}\right)^{\sum^n_{j=1}n_{ij}^{(1)}} \right]
$$

對 L1 取 log，loglikelihood 命名為 LL1，並移除掉與待估計參數無關的部份:
$$
LL1 = \sum^m_{i=1}\left[\sum^n_{j=1}n_{ij}^{(1)}log(\Lambda_{j1})+log(\Gamma(\alpha_1 + \sum^n_{j=1}n_{ij}^{(1)})) - log(\Gamma(\alpha_1)) - log(\Gamma(\sum^n_{j=1}n_{ij}^{(1)} + 1)) + \alpha_1 log(\alpha_1) + (\beta_1' Z)\sum^n_{j=1}n_{ij}^{(1)} - (\alpha_1 + \sum^n_{j=1}n_{ij}^{(1)})log(\alpha_1 + e^{\beta_1' Z}\sum^n_{j=1}\Lambda_{j1})\right]
$$

In [4]:
LL1 = function(par, data){
    # par = c(alpha1, beta10, beta11, beta12, dl11, dl21, dl31, dl41, dl51)

    alpha1 = par[1]
    beta10 = par[2]
    beta11 = par[3]
    beta12 = par[4]
    dl11 = par[5]
    dl21 = par[6]
    dl31 = par[7]
    dl41 = par[8]
    dl51 = par[9]
    
    ni11 = data[, 1]
    ni21 = data[, 2]
    ni31 = data[, 3]
    ni41 = data[, 4]
    ni51 = data[, 5]
    z1 = data[, 11]
    z2 = data[, 12]

    nij1.sum.j = ni11 + ni21 + ni31 + ni41 + ni51
    dlj1.sum.j = dl11 + dl21 + dl31 + dl41 + dl51
    
    
    C1 = sum(ni11 * log(dl11) + ni21 * log(dl21) + ni31 * log(dl31) + ni41 * log(dl41) + ni51 * log(dl51))
    C2 = log(gamma(alpha1 + nij1.sum.j))
    C3 = log(gamma(alpha1))
    C4 = log(gamma(nij1.sum.j + 1))
    C5 = alpha1 * log(alpha1)
    C6 = (beta10 + beta11 * z1 + beta12 * z2) * nij1.sum.j
    C7 = (alpha1 + nij1.sum.j) * log(alpha1 + exp(beta10 + beta11 * z1 + beta12 * z2) * dlj1.sum.j)

    return ((-1) * sum(C1 + C2 - C3 - C4 + C5 + C6 - C7))
}

$$
\frac{\partial}{\partial \alpha_1}LL1 = \sum_{i=1}^m\left\{\frac{[\Gamma(\alpha_1 + \sum^n_{j=1}n_{ij}^{(1)})]'}{\Gamma(\alpha_1 + \sum^n_{j=1}n_{ij}^{(1)})} - \frac{[\Gamma(\alpha_1)]'}{\Gamma(\alpha_1)} + log(\alpha_1) + 1 - log(\alpha_1 + e^{\beta_1' Z}\sum^n_{j=1}\Lambda_{j1}) - \frac{\alpha_1 + \sum^n_{j=1}n_{ij}^{(1)}}{\alpha_1 + e^{\beta_1' Z}\sum^n_{j=1}\Lambda_{j1}}\right\}
$$

$$
\frac{\partial}{\partial \beta_{10}} LL1 = \sum^m_{i=1}\left\{\sum^n_{j=1}n_{ij}^{(1)} - (\alpha_1 + \sum^n_{j=1}n_{ij}^{(1)})\frac{e^{\beta_1'Z}\sum^n_{j=1}\Lambda_{j1}}{\alpha_1 + e^{\beta_1'Z}\sum^n_{j=1}\Lambda_{j1}}\right\}
$$

$$
\frac{\partial}{\partial \beta_{11}} LL1 = \sum_{i=1}^m\left\{z_1\sum^n_{j=1}n_{ij}^{(1)} - (\alpha_1 + \sum^n_{j=1}n_{ij}^{(1)})\frac{z_1 e^{\beta_1'Z}\sum^n_{j=1}\Lambda_{j1}}{\alpha_1 + e^{\beta_1'Z}\sum^n_{j=1}\Lambda_{j1}}\right\}
$$

$$
\frac{\partial}{\partial \beta_{12}} LL1 = \sum_{i=1}^m\left\{z_2 \sum^n_{j=1}n_{ij}^{(1)} - (\alpha_1 + \sum^n_{j=1}n_{ij}^{(1)})\frac{z_2 e^{\beta_1'Z}\sum^n_{j=1}\Lambda_{j1}}{\alpha_1 + e^{\beta_1'Z}\sum^n_{j=1}\Lambda_{j1}}\right\}
$$

$$
\frac{\partial}{\partial \Lambda_{11}} LL1 = \sum_{i=1}^m\left\{\frac{n_{i1}^{(1)}}{\Lambda_{11}} - (\alpha_1 + \sum^n_{j=1}n_{ij}^{(1)})\frac{e^{\beta_1'Z}}{\alpha_1 + e^{\beta_1' Z}\sum^n_{j=1}\Lambda_{j1}}\right\}
$$

$$
\frac{\partial}{\partial \Lambda_{21}} LL1 = \sum_{i=1}^m\left\{\frac{n_{i2}^{(1)}}{\Lambda_{21}} - (\alpha_1 + \sum^n_{j=1}n_{ij}^{(1)})\frac{e^{\beta_1'Z}}{\alpha_1 + e^{\beta_1' Z}\sum^n_{j=1}\Lambda_{j1}}\right\}
$$

$$
\frac{\partial}{\partial \Lambda_{31}} LL1 = \sum_{i=1}^m\left\{\frac{n_{i3}^{(1)}}{\Lambda_{31}}  - (\alpha_1 + \sum^n_{j=1}n_{ij}^{(1)})\frac{e^{\beta_1'Z}}{\alpha_1 + e^{\beta_1' Z}\sum^n_{j=1}\Lambda_{j1}}\right\}
$$

$$
\frac{\partial}{\partial \Lambda_{41}} LL1 = \sum_{i=1}^m\left\{\frac{n_{i4}^{(1)}}{\Lambda_{41}} - (\alpha_1 + \sum^n_{j=1}n_{ij}^{(1)})\frac{e^{\beta_1'Z}}{\alpha_1 + e^{\beta_1' Z}\sum^n_{j=1}\Lambda_{j1}}\right\}
$$

$$
\frac{\partial}{\partial \Lambda_{51}} LL1 = \sum_{i=1}^m\left\{\frac{n_{i5}^{(1)}}{\Lambda_{51}} - (\alpha_1 + \sum^n_{j=1}n_{ij}^{(1)})\frac{e^{\beta_1'Z}}{\alpha_1 + e^{\beta_1' Z}\sum^n_{j=1}\Lambda_{j1}}\right\}
$$

In [5]:
gra1 = function(par, data){
    # par = c(alpha1, beta10, beta11, beta12, dl11, dl21, dl31, dl41, dl51)
    
    alpha1 = par[1]
    beta10 = par[2]
    beta11 = par[3]
    beta12 = par[4]
    dl11 = par[5]
    dl21 = par[6]
    dl31 = par[7]
    dl41 = par[8]
    dl51 = par[9]
    
    ni11 = data[, 1]
    ni21 = data[, 2]
    ni31 = data[, 3]
    ni41 = data[, 4]
    ni51 = data[, 5]
    z1 = data[, 11]
    z2 = data[, 12]

    nij1.sum.j = ni11 + ni21 + ni31 + ni41 + ni51
    dlj1.sum.j = dl11 + dl21 + dl31 + dl41 + dl51

    linear_comp = beta10 + beta11 * z1 + beta12 * z2
    linear_comp_exp = exp(linear_comp)
    s1 = alpha1 + nij1.sum.j
    s2 = alpha1 + linear_comp_exp * dlj1.sum.j
    s3 = linear_comp_exp * dlj1.sum.j

    p_alpha1 = sum(digamma(alpha1 + nij1.sum.j) - digamma(alpha1) + log(alpha1) + 1 - log(s2) - (s1 / s2))
    p_beta10 = sum(nij1.sum.j - (s1 * s3 / s2))
    p_beta11 = sum(z1 * nij1.sum.j - z1 * (s1 * s3 / s2))
    p_beta12 = sum(z2 * nij1.sum.j - z2 * (s1 * s3 / s2))
    p_dl11 = sum((ni11/dl11) - (s1 * linear_comp_exp / s2))
    p_dl21 = sum((ni21/dl21) - (s1 * linear_comp_exp / s2))
    p_dl31 = sum((ni31/dl31) - (s1 * linear_comp_exp / s2))
    p_dl41 = sum((ni41/dl41) - (s1 * linear_comp_exp / s2))
    p_dl51 = sum((ni51/dl51) - (s1 * linear_comp_exp / s2))
    
    c(p_alpha1, p_beta10, p_beta11, p_beta12, p_dl11, p_dl21, p_dl31, p_dl41, p_dl51)
}

In [6]:
options(warn = -1)
dt = generate.data(N = 300, alpha = c(3, 2), beta1 = c(0.5, 0.5, -0.5), beta2 = c(0.5, -0.5, 0.5), omega = 0.5)
s = c(3+runif(1, -0.05, 0.05), 0.5+runif(1, -0.05, 0.05), 0.5+runif(1, -0.05, 0.05),
      -0.5+runif(1, -0.05, 0.05), 0.05+runif(1, -0.01, 0.01), 0.05+runif(1, -0.01, 0.01),
      0.05+runif(1, -0.01, 0.01), 0.05+runif(1, -0.01, 0.01), 0.05+runif(1, -0.01, 0.01))
p = unlist(optim(par = s, fn = LL1, data = dt, gr = gra1, method = "BFGS")$par)
print(p)

[1]  3.01854415  0.48443523  0.49576988 -0.45080316  0.04189317  0.05074926
[7]  0.05946995  0.05647129  0.04431287


再估計第二種姿勢的部份,其部份的 likelihood 命名為 L2：

其相關的部份有
$$
L2 = \prod^m_{i=1}\left[ \binom{\sum^n_{j = 1}n_{ij}^{(2)}}{n_{i1}^{(2)}, \cdots, n_{in}^{(2)}}\prod^n_{j=1}p_{j2}^{n_{ij}^{(2)}} \frac{\Gamma(\alpha_2 + \sum^n_{j=1}n_{ij}^{(2)})}{\Gamma(\alpha_2)(\sum^n_{j=1}n_{ij}^{(2)})!}\left(\frac{\alpha_2}{\alpha_2 + e^{\beta_2'Z }\sum^n_{j=1}\Lambda_{j2}}\right)^{\alpha_2} \left(\frac{e^{\beta_2'Z }\sum^n_{j=1}\Lambda_{j2}}{\alpha_2 + e^{\beta_2'Z }\sum^n_{j=1}\Lambda_{j2}}\right)^{\sum^n_{j=1}n_{ij}^{(2)}} \right]
$$

對 L2 取 log，loglikelihood 命名為 LL2，並移除掉與待估計參數無關的部份:
$$
LL2 = \sum^m_{i=1}\left[\sum^n_{j=1}n_{ij}^{(2)}log(\Lambda_{j2})+log(\Gamma(\alpha_2 + \sum^n_{j=1}n_{ij}^{(2)})) - log(\Gamma(\alpha_2)) - log(\Gamma(\sum^n_{j=1}n_{ij}^{(2)} + 1)) + \alpha_2 log(\alpha_2) + (\beta_2' Z)\sum^n_{j=1}n_{ij}^{(2)} - (\alpha_2 + \sum^n_{j=1}n_{ij}^{(2)})log(\alpha_2 + e^{\beta_2' Z}\sum^n_{j=1}\Lambda_{j2})\right]
$$

In [7]:
LL2 = function(par, data){
    # par = c(alpha2, beta20, beta21, beta22, dl12, dl22, dl32, dl42, dl52)

    alpha2 = par[1]
    beta20 = par[2]
    beta21 = par[3]
    beta22 = par[4]
    dl12 = par[5]
    dl22 = par[6]
    dl32 = par[7]
    dl42 = par[8]
    dl52 = par[9]
    
    ni12 = data[, 6]
    ni22 = data[, 7]
    ni32 = data[, 8]
    ni42 = data[, 9]
    ni52 = data[, 10]
    z1 = data[, 11]
    z2 = data[, 12]

    nij2.sum.j = ni12 + ni22 + ni32 + ni42 + ni52
    dlj2.sum.j = dl12 + dl22 + dl32 + dl42 + dl52
    
    
    C1 = sum(ni12 * log(dl12) + ni22 * log(dl22) + ni32 * log(dl32) + ni42 * log(dl42) + ni52 * log(dl52))
    C2 = log(gamma(alpha2 + nij2.sum.j))
    C3 = log(gamma(alpha2))
    C4 = log(gamma(nij2.sum.j + 1))
    C5 = alpha2 * log(alpha2)
    C6 = (beta20 + beta21 * z1 + beta22 * z2) * nij2.sum.j
    C7 = (alpha2 + nij2.sum.j) * log(alpha2 + exp(beta20 + beta21 * z1 + beta22 * z2) * dlj2.sum.j)

    return ((-1) * sum(C1 + C2 - C3 - C4 + C5 + C6 - C7))
}

$$
\frac{\partial}{\partial \alpha_2}LL2 = \sum_{i=1}^m\left\{\frac{[\Gamma(\alpha_2 + \sum^n_{j=1}n_{ij}^{(2)})]'}{\Gamma(\alpha_2 + \sum^n_{j=1}n_{ij}^{(2)})} - \frac{[\Gamma(\alpha_2)]'}{\Gamma(\alpha_2)} + log(\alpha_2) + 1 - log(\alpha_2 + e^{\beta_2' Z}\sum^n_{j=1}\Lambda_{j2}) - \frac{\alpha_2 + \sum^n_{j=1}n_{ij}^{(2)}}{\alpha_2 + e^{\beta_2' Z}\sum^n_{j=1}\Lambda_{j2}}\right\}
$$

$$
\frac{\partial}{\partial \beta_{20}} LL2 = \left\{\sum^n_{j=1}n_{ij}^{(2)} - (\alpha_2 + \sum^n_{j=1}n_{ij}^{(2)})\frac{e^{\beta_2'Z}\sum^n_{j=1}\Lambda_{j2}}{\alpha_2 + e^{\beta_2'Z}\sum^n_{j=1}\Lambda_{j2}}\right\}
$$

$$
\frac{\partial}{\partial \beta_{21}} LL2 = \sum_{i=1}^m\left\{z_1\sum^n_{j=1}n_{ij}^{(2)} - (\alpha_2 + \sum^n_{j=1}n_{ij}^{(2)})\frac{z_1 e^{\beta_2'Z}\sum^n_{j=1}\Lambda_{j2}}{\alpha_2 + e^{\beta_2'Z}\sum^n_{j=1}\Lambda_{j2}}\right\}
$$

$$
\frac{\partial}{\partial \beta_{22}} LL2 = \sum_{i=1}^m\left\{z_2 \sum^n_{j=1}n_{ij}^{(2)} - (\alpha_2 + \sum^n_{j=1}n_{ij}^{(2)})\frac{z_2 e^{\beta_2'Z}\sum^n_{j=1}\Lambda_{j2}}{\alpha_2 + e^{\beta_2'Z}\sum^n_{j=1}\Lambda_{j2}}\right\}
$$

$$
\frac{\partial}{\partial \Lambda_{12}} LL2 = \sum_{i=1}^m\left\{\frac{n_{i1}^{(2)}}{\Lambda_{12}} - (\alpha_2 + \sum^n_{j=1}n_{ij}^{(2)})\frac{e^{\beta_2'Z}}{\alpha_2 + e^{\beta_2' Z}\sum^n_{j=1}\Lambda_{j2}}\right\}
$$

$$
\frac{\partial}{\partial \Lambda_{22}} LL2 = \sum_{i=1}^m\left\{\frac{n_{i2}^{(2)}}{\Lambda_{22}} - (\alpha_2 + \sum^n_{j=1}n_{ij}^{(2)})\frac{e^{\beta_2'Z}}{\alpha_2 + e^{\beta_2' Z}\sum^n_{j=1}\Lambda_{j2}}\right\}
$$

$$
\frac{\partial}{\partial \Lambda_{32}} LL2 = \sum_{i=1}^m\left\{\frac{n_{i3}^{(2)}}{\Lambda_{32}} - (\alpha_2 + \sum^n_{j=1}n_{ij}^{(2)})\frac{e^{\beta_2'Z}}{\alpha_2 + e^{\beta_2' Z}\sum^n_{j=1}\Lambda_{j2}}\right\}
$$

$$
\frac{\partial}{\partial \Lambda_{42}} LL2 = \sum_{i=1}^m\left\{\frac{n_{i4}^{(2)}}{\Lambda_{42}} - (\alpha_2 + \sum^n_{j=1}n_{ij}^{(2)})\frac{e^{\beta_2'Z}}{\alpha_2 + e^{\beta_2' Z}\sum^n_{j=1}\Lambda_{j2}}\right\}
$$

$$
\frac{\partial}{\partial \Lambda_{52}} LL2 = \sum_{i=1}^m\left\{\frac{n_{i5}^{(2)}}{\Lambda_{52}} - (\alpha_2 + \sum^n_{j=1}n_{ij}^{(2)})\frac{e^{\beta_2'Z}}{\alpha_2 + e^{\beta_2' Z}\sum^n_{j=1}\Lambda_{j2}}\right\}
$$

In [8]:
gra2 = function(par, data){
    # par = c(alpha2, beta20, beta21, beta22, dl11, dl21, dl31, dl41, dl51)
    
    alpha2 = par[1]
    beta20 = par[2]
    beta21 = par[3]
    beta22 = par[4]
    dl12 = par[5]
    dl22 = par[6]
    dl32 = par[7]
    dl42 = par[8]
    dl52 = par[9]
    
    ni12 = data[, 6]
    ni22 = data[, 7]
    ni32 = data[, 8]
    ni42 = data[, 9]
    ni52 = data[, 10]
    z1 = data[, 11]
    z2 = data[, 12]

    nij2.sum.j = ni12 + ni22 + ni32 + ni42 + ni52
    dlj2.sum.j = dl12 + dl22 + dl32 + dl42 + dl52

    linear_comp = beta20 + beta21 * z1 + beta22 * z2
    linear_comp_exp = exp(linear_comp)
    s1 = alpha2 + nij2.sum.j
    s2 = alpha2 + linear_comp_exp * dlj2.sum.j
    s3 = linear_comp_exp * dlj2.sum.j

    p_alpha2 = sum(digamma(alpha2 + nij2.sum.j) - digamma(alpha2) + log(alpha2) + 1 - log(s2) - (s1 / s2))
    p_beta20 = sum(nij2.sum.j - (s1 * s3 / s2))
    p_beta21 = sum(z1 * nij2.sum.j - z1 * (s1 * s3 / s2))
    p_beta22 = sum(z2 * nij2.sum.j - z2 * (s1 * s3 / s2))
    p_dl12 = sum((ni12/dl12) - (s1 * linear_comp_exp / s2))
    p_dl22 = sum((ni22/dl22) - (s1 * linear_comp_exp / s2))
    p_dl32 = sum((ni32/dl32) - (s1 * linear_comp_exp / s2))
    p_dl42 = sum((ni42/dl42) - (s1 * linear_comp_exp / s2))
    p_dl52 = sum((ni52/dl52) - (s1 * linear_comp_exp / s2))
    
    c(p_alpha2, p_beta20, p_beta21, p_beta22, p_dl12, p_dl22, p_dl32, p_dl42, p_dl52)
}

In [9]:
#options(warn = -1)
dt = generate.data(N=300, alpha = c(3, 2), beta1 = c(0.5, 0.5, -0.5), beta2 = c(0.5, -0.5, 0.5), omega = 0.5)
s = c(2+runif(1, -0.05, 0.05), 0.5+runif(1, -0.05, 0.05), -0.5+runif(1, -0.05, 0.05),
      0.5+runif(1, -0.05, 0.05), 0.025+runif(1, -0.01, 0.01), 0.075+runif(1, -0.01, 0.01),
      0.125+runif(1, -0.01, 0.01), 0.175+runif(1, -0.01, 0.01), 0.225+runif(1, -0.01, 0.01))
p = unlist(optim(par = s, fn = LL2, data = dt, gr = gra2, method = "BFGS")$par)
print(p)

[1]  1.99620163  0.47385872 -0.54558978  0.53175474  0.02563889  0.07786163
[7]  0.11990741  0.16741545  0.21883282


In [10]:
LL.all = function(par, known, data){
    # par = c(alpha1, beta10, beta11, beta12, dl11, dl21, dl31, dl41, dl51, alpha2, beta20, beta21, beta22, dl11, dl21, dl31, dl41, dl51)
    omega = par[1]
    alpha1 = known[1]
    beta10 = known[2]
    beta11 = known[3]
    beta12 = known[4]
    dl11 = known[5]
    dl21 = known[6]
    dl31 = known[7]
    dl41 = known[8]
    dl51 = known[9]
    alpha2 = known[10]
    beta20 = known[11]
    beta21 = known[12]
    beta22 = known[13]
    dl12 = known[14]
    dl22 = known[15]
    dl32 = known[16]
    dl42 = known[17]
    dl52 = known[18]
    ni11 = data[, 1]
    ni21 = data[, 2]
    ni31 = data[, 3]
    ni41 = data[, 4]
    ni51 = data[, 5]
    ni12 = data[, 6]
    ni22 = data[, 7]
    ni32 = data[, 8]
    ni42 = data[, 9]
    ni52 = data[, 10]
    z1 = data[, 11]
    z2 = data[, 12]

    nij1.sum.j = ni11 + ni21 + ni31 + ni41 + ni51
    dlj1.sum.j = dl11 + dl21 + dl31 + dl41 + dl51
    nij2.sum.j = ni12 + ni22 + ni32 + ni42 + ni52
    dlj2.sum.j = dl12 + dl22 + dl32 + dl42 + dl52
    linear1 = beta10 + beta11 * z1 + beta12 * z2
    linear2 = beta20 + beta21 * z1 + beta22 * z2
    exp1 = exp(beta10 + beta11 * z1 + beta12 * z2)
    exp2 = exp(beta20 + beta21 * z1 + beta22 * z2)

    C1 = sum(ni11 * log(dl11) + ni21 * log(dl21) + ni31 * log(dl31) + ni41 * log(dl41) + ni51 * log(dl51))
    C2 = log(gamma(alpha1 + nij1.sum.j))
    C3 = log(gamma(alpha1))
    C4 = log(gamma(nij1.sum.j + 1))
    C5 = alpha1 * log(alpha1)
    C6 = linear1 * nij1.sum.j
    C7 = (alpha1 + nij1.sum.j) * log(alpha1 + exp1 * dlj1.sum.j)

    C8 = sum(ni12 * log(dl12) + ni22 * log(dl22) + ni32 * log(dl32) + ni42 * log(dl42) + ni52 * log(dl52))
    C9 = log(gamma(alpha2 + nij2.sum.j))
    C10 = log(gamma(alpha2))
    C11 = log(gamma(nij2.sum.j + 1))
    C12 = alpha2 * log(alpha2)
    C13 = linear2 * nij2.sum.j
    C14 = (alpha2 + nij2.sum.j) * log(alpha2 + exp2 * dlj2.sum.j)

    C151 = ((alpha1 + exp1 * dlj1.sum.j)/(alpha1 + exp1 * dlj1.sum.j + 1))^(alpha1 + nij1.sum.j)
    C152 = (alpha1/(alpha1 + 1))^(alpha1)
    C153 = ((alpha2 + exp2 * dlj2.sum.j)/(alpha2 + exp2 * dlj2.sum.j + 1))^(alpha2 + nij2.sum.j)
    C154 = (alpha2/(alpha2 + 1))^(alpha2)
    C15 = 1 + omega * (C151 - C152) * (C153 - C154)
    return ((-1)*sum(C1 + C2 - C3 - C4 + C5 + C6 - C7 + C8 + C9 - C10 - C11 + C12 + C13 - C14 + C15))
}

In [11]:
options(warn = -1)
dt = generate.data(N=300, alpha = c(3, 2), beta1 = c(0.5, 0.5, -0.5), beta2 = c(0.5, -0.5, 0.5), omega = 0.5)
s1 = c(3+runif(1, -0.05, 0.05), 0.5+runif(1, -0.05, 0.05), 0.5+runif(1, -0.05, 0.05),
      -0.5+runif(1, -0.05, 0.05), 0.05+runif(1, -0.01, 0.01), 0.05+runif(1, -0.01, 0.01),
      0.05+runif(1, -0.01, 0.01), 0.05+runif(1, -0.01, 0.01), 0.05+runif(1, -0.01, 0.01))
s2 = c(2+runif(1, -0.05, 0.05), 0.5+runif(1, -0.05, 0.05), -0.5+runif(1, -0.05, 0.05),
      0.5+runif(1, -0.05, 0.05), 0.025+runif(1, -0.01, 0.01), 0.075+runif(1, -0.01, 0.01),
      0.125+runif(1, -0.01, 0.01), 0.175+runif(1, -0.01, 0.01), 0.225+runif(1, -0.01, 0.01))
p1 = unlist(optim(par = s1, fn = LL1, data = dt, gr = gra1, method = "BFGS")$par)
p2 = unlist(optim(par = s2, fn = LL2, data = dt, gr = gra2, method = "BFGS")$par)
p = c(p1, p2)
ome = optimize(f = LL.all, interval = c(0.5+runif(1, 0, 0.1), 0.5+runif(1, -0.1, 0)), known = p, data = dt)$minimum
p = c(p, ome)
print(p)

 [1]  2.95679339  0.48743544  0.47531324 -0.47920380  0.04859972  0.04459184
 [7]  0.05088461  0.05961219  0.04696731  1.96155360  0.45161540 -0.45767829
[13]  0.48945056  0.02329028  0.06937188  0.13016704  0.17917257  0.21855376
[19]  0.44942232


#### 模擬一千次

In [12]:
options(warn = -1)
param.all = c()
for (i in 1:1000){
    dt = generate.data(N = 300, alpha = c(3, 2), beta1 = c(0.5, 0.5, -0.5), beta2 = c(0.5, -0.5, 0.5), omega = 0.5)
    s1 = c(3+runif(1, -0.05, 0.05), 0.5+runif(1, -0.05, 0.05), 0.5+runif(1, -0.05, 0.05),
        -0.5+runif(1, -0.05, 0.05), 0.05+runif(1, -0.01, 0.01), 0.05+runif(1, -0.01, 0.01),
        0.05+runif(1, -0.01, 0.01), 0.05+runif(1, -0.01, 0.01), 0.05+runif(1, -0.01, 0.01))
    s2 = c(2+runif(1, -0.05, 0.05), 0.5+runif(1, -0.05, 0.05), -0.5+runif(1, -0.05, 0.05),
        0.5+runif(1, -0.05, 0.05), 0.025+runif(1, -0.01, 0.01), 0.075+runif(1, -0.01, 0.01),
        0.125+runif(1, -0.01, 0.01), 0.175+runif(1, -0.01, 0.01), 0.225+runif(1, -0.01, 0.01))
    p1 = unlist(optim(par = s1, fn = LL1, data = dt, gr = gra1, method = "BFGS")$par)
    p2 = unlist(optim(par = s2, fn = LL2, data = dt, gr = gra2, method = "BFGS")$par)
    ome = optimize(f = LL.all, interval = c(0.5+runif(1, 0, 0.1), 0.5+runif(1, -0.1, 0)), known = p, data = dt)$minimum
    p = c(p1, p2, ome)
    param.all = rbind(param.all, p)
}

print(param.all)

      [,1]       [,2]        [,3]       [,4]         [,5]         [,6]
p 3.000735  0.4689035   0.4998714 -0.4503737 5.879194e-02 4.437596e-02
p 3.037498  0.5259262   0.4940518 -0.4666966 4.631108e-02 5.708951e-02
p 2.971361  0.5153382   0.5031002 -0.5060204 4.536132e-02 5.389033e-02
p 3.012773  0.5271197   0.5023307 -0.5223879 5.806724e-02 4.325864e-02
p 2.973191  0.4780491   0.5033970 -0.4640071 4.021828e-02 4.624981e-02
p 3.035981  0.4972287   0.5392295 -0.4859729 5.924383e-02 4.900303e-02
p 2.987399  0.5226324   0.5440928 -0.5070897 4.493724e-02 4.551318e-02
p 2.975261  0.5472681   0.4710112 -0.4383564 1.225240e-01 2.949439e-02
p 2.865967 25.2595740  21.7127840 10.5920738 1.330374e+02 3.760160e+01
p 3.025003  0.4626873   0.5484345 -0.5003116 4.686261e-02 4.965156e-02
p 3.017893  0.5179233   0.5133158 -0.5203477 4.461541e-02 5.336133e-02
p 2.958522  0.5255834   0.4785424 -0.4574030 4.170547e-02 4.207383e-02
p 2.965012  0.5459248   0.4829453 -0.5226222 4.244273e-02 5.874160e-02
p 3.04

In [13]:
index = c()
for (i in 1:length(param.all[, 1])){
    l = c(param.all[i, c(5:9, 14:18)] > 1)
    if (TRUE %in% l){
        index = c(index, i)
    }else{
        next
    }
}

print(index)

 [1]   9  17  29  74  99 100 130 141 149 185 197 205 230 257 262 271 277 295 297
[20] 322 329 350 352 367 370 376 407 436 442 460 489 498 536 540 547 560 589 612
[39] 632 636 648 649 666 687 698 700 742 745 752 753 758 759 785 799 811 846 861
[58] 890 892 898 948 965 993


In [14]:
param.all = param.all[-c(index), ]
colnames(param.all) = c("alpha1", "beta10", "beta11", "beta12", "dlambda11", "dlambda21", "dlambda31", "dlambda41", "dlambda51",
                        "alpha2", "beta20", "beta21", "beta22", "dlambda12", "dlambda22", "dlambda32", "dlambda42", "dlambda52",
                        "omega")
print(head(param.all))

    alpha1    beta10    beta11     beta12  dlambda11  dlambda21  dlambda31
p 3.000735 0.4689035 0.4998714 -0.4503737 0.05879194 0.04437596 0.05393379
p 3.037498 0.5259262 0.4940518 -0.4666966 0.04631108 0.05708951 0.04436079
p 2.971361 0.5153382 0.5031002 -0.5060204 0.04536132 0.05389033 0.04463168
p 3.012773 0.5271197 0.5023307 -0.5223879 0.05806724 0.04325864 0.05586975
p 2.973191 0.4780491 0.5033970 -0.4640071 0.04021828 0.04624981 0.05464092
p 3.035981 0.4972287 0.5392295 -0.4859729 0.05924383 0.04900303 0.04057136
   dlambda41  dlambda51   alpha2    beta20     beta21    beta22  dlambda12
p 0.05500159 0.04306453 2.025655 0.5361602 -0.5505194 0.5286233 0.02174342
p 0.06753955 0.04817143 1.991764 0.5741401 -0.4607070 0.5120205 0.79438351
p 0.05442080 0.04569239 2.047053 0.4636684 -0.5019227 0.4993540 0.03235563
p 0.04567601 0.04728234 2.042563 0.5261661 -0.4624468 0.5117905 0.02537594
p 0.05432086 0.05963137 1.974179 0.4790423 -0.4887150 0.4889226 0.02116139
p 0.05576656 0.03608324 2

In [15]:
# mean
print(apply(param.all, 2, mean))
cat("==================================================================================================\n")
# bias
print(apply(param.all, 2, mean) - c(3, 0.5, 0.5, -0.5, 0.05, 0.05, 0.05, 0.05, 0.05, 2, 0.5, -0.5, 0.5, 0.025, 0.075, 0.125, 0.175, 0.225, 0.5))
cat("==================================================================================================\n")
# standard deviation
print(apply(param.all, 2, sd))

     alpha1      beta10      beta11      beta12   dlambda11   dlambda21 
 3.00007188  0.50232319  0.50014283 -0.49888221  0.05412494  0.05638775 
  dlambda31   dlambda41   dlambda51      alpha2      beta20      beta21 
 0.05499627  0.05594585  0.05613668  1.99976175  0.50259558 -0.50013136 
     beta22   dlambda12   dlambda22   dlambda32   dlambda42   dlambda52 
 0.50025397  0.05520321  0.08780982  0.13149813  0.17714609  0.22618466 
      omega 
 0.50221736 
       alpha1        beta10        beta11        beta12     dlambda11 
 7.187673e-05  2.323189e-03  1.428335e-04  1.117791e-03  4.124944e-03 
    dlambda21     dlambda31     dlambda41     dlambda51        alpha2 
 6.387750e-03  4.996270e-03  5.945851e-03  6.136685e-03 -2.382472e-04 
       beta20        beta21        beta22     dlambda12     dlambda22 
 2.595579e-03 -1.313554e-04  2.539655e-04  3.020321e-02  1.280982e-02 
    dlambda32     dlambda42     dlambda52         omega 
 6.498130e-03  2.146095e-03  1.184660e-03  2.217361e-

---

---

---

---

與上面不同，這邊將部分有範圍的參數全部採用exponential，增加估計穩定度。
+ $\alpha_1 = exp(\alpha_1')$
+ $\Lambda_{j1} = exp(\Lambda_{j1}')$
  + 為了方便，以下的prime全部省略。

$$
LL1 = \sum^m_{i=1}\left[\sum^n_{j=1}n_{ij}^{(1)}\Lambda_{j1}+log(\Gamma(e^{\alpha_1} + \sum^n_{j=1}n_{ij}^{(1)})) - log(\Gamma(e^{\alpha_1})) - log(\Gamma(\sum^n_{j=1}n_{ij}^{(1)} + 1)) + \alpha_1 e^{\alpha_1} + (\beta_1' Z)\sum^n_{j=1}n_{ij}^{(1)} - (e^{\alpha_1} + \sum^n_{j=1}n_{ij}^{(1)})log(e^{\alpha_1} + e^{\beta_1' Z}\sum^n_{j=1}e^{\Lambda_{j1}})\right]
$$

In [16]:
LL1.new = function(par, data){
    # par = c(alpha1, beta10, beta11, beta12, dl11, dl21, dl31, dl41, dl51)

    alpha1 = par[1]
    beta10 = par[2]
    beta11 = par[3]
    beta12 = par[4]
    dl11 = par[5]
    dl21 = par[6]
    dl31 = par[7]
    dl41 = par[8]
    dl51 = par[9]
    
    ni11 = data[, 1]
    ni21 = data[, 2]
    ni31 = data[, 3]
    ni41 = data[, 4]
    ni51 = data[, 5]
    z1 = data[, 11]
    z2 = data[, 12]

    nij1.sum.j = ni11 + ni21 + ni31 + ni41 + ni51
    dlj1.sum.j = exp(dl11) + exp(dl21) + exp(dl31) + exp(dl41) + exp(dl51)
    
    
    C1 = sum(ni11 * dl11 + ni21 * dl21 + ni31 * dl31 + ni41 * dl41 + ni51 * dl51)
    C2 = log(gamma(exp(alpha1) + nij1.sum.j))
    C3 = log(gamma(exp(alpha1)))
    C4 = log(gamma(nij1.sum.j + 1))
    C5 = alpha1 * exp(alpha1)
    C6 = (beta10 + beta11 * z1 + beta12 * z2) * nij1.sum.j
    C7 = (exp(alpha1) + nij1.sum.j) * log(exp(alpha1) + exp(beta10 + beta11 * z1 + beta12 * z2) * dlj1.sum.j)

    return ((-1) * sum(C1 + C2 - C3 - C4 + C5 + C6 - C7))
}

$$
\frac{\partial}{\partial \alpha_1}LL1 = \sum_{i=1}^m\left\{\frac{[\Gamma(e^{\alpha_1} + \sum^n_{j=1}n_{ij}^{(1)})]'}{\Gamma(e^{\alpha_1} + \sum^n_{j=1}n_{ij}^{(1)})} - \frac{[\Gamma(e^{\alpha_1})]'}{\Gamma(e^{\alpha_1})} + e^{\alpha_1} + \alpha_1 e^{\alpha1} - e^{\alpha_1}log(e^{\alpha_1} + e^{\beta_1' Z}\sum^n_{j=1}e^{\Lambda_{j1}}) - \frac{e^{\alpha_1}(e^{\alpha_1} + \sum^n_{j=1}n_{ij}^{(1)})}{\alpha_1 + e^{\beta_1' Z}\sum^n_{j=1}\Lambda_{j1}}\right\}
$$

$$
\frac{\partial}{\partial \beta_{10}} LL1 = \sum^m_{i=1}\left\{\sum^n_{j=1}n_{ij}^{(1)} - (e^{\alpha_1} + \sum^n_{j=1}n_{ij}^{(1)})\frac{e^{\beta_1'Z}\sum^n_{j=1}e^{\Lambda_{j1}}}{e^{\alpha_1} + e^{\beta_1'Z}\sum^n_{j=1}e^{\Lambda_{j1}}}\right\}
$$

$$
\frac{\partial}{\partial \beta_{11}} LL1 = \sum^m_{i=1}\left\{z_1\sum^n_{j=1}n_{ij}^{(1)} - (e^{\alpha_1} + \sum^n_{j=1}n_{ij}^{(1)})\frac{z_1 e^{\beta_1'Z}\sum^n_{j=1}e^{\Lambda_{j1}}}{e^{\alpha_1} + e^{\beta_1'Z}\sum^n_{j=1}e^{\Lambda_{j1}}}\right\}
$$

$$
\frac{\partial}{\partial \beta_{12}} LL1 = \sum^m_{i=1}\left\{z_2\sum^n_{j=1}n_{ij}^{(1)} - (e^{\alpha_1} + \sum^n_{j=1}n_{ij}^{(1)})\frac{z_2 e^{\beta_1'Z}\sum^n_{j=1}e^{\Lambda_{j1}}}{e^{\alpha_1} + e^{\beta_1'Z}\sum^n_{j=1}e^{\Lambda_{j1}}}\right\}
$$

$$
\frac{\partial}{\partial \Lambda_{j1}} LL1 = \sum^m_{i=1}\left\{n_{ij}^{(1)} - (e^{\alpha_1} + \sum^n_{j=1}n_{ij}^{(1)})\frac{e^{\beta_1'Z }e^{\Lambda_{j1}}}{e^{\alpha_1} + e^{\beta_1' Z}\sum^n_{j=1}e^{\Lambda_{j1}}} \right\}
$$
where j = 1, 2, 3, 4, 5

In [17]:
gra1.new = function(par, data){
    # par = c(alpha1, beta10, beta11, beta12, dl11, dl21, dl31, dl41, dl51)
    
    alpha1 = par[1]
    beta10 = par[2]
    beta11 = par[3]
    beta12 = par[4]
    dl11 = par[5]
    dl21 = par[6]
    dl31 = par[7]
    dl41 = par[8]
    dl51 = par[9]
    
    ni11 = data[, 1]
    ni21 = data[, 2]
    ni31 = data[, 3]
    ni41 = data[, 4]
    ni51 = data[, 5]
    z1 = data[, 11]
    z2 = data[, 12]

    nij1.sum.j = ni11 + ni21 + ni31 + ni41 + ni51
    dlj1.sum.j = exp(dl11) + exp(dl21) + exp(dl31) + exp(dl41) + exp(dl51)

    linear_comp = beta10 + beta11 * z1 + beta12 * z2
    linear_comp_exp = exp(linear_comp)
    s1 = exp(alpha1) + nij1.sum.j
    s2 = exp(alpha1) + linear_comp_exp * dlj1.sum.j
    s3 = linear_comp_exp * dlj1.sum.j

    p_alpha1 = sum(digamma(exp(alpha1) + nij1.sum.j) - digamma(exp(alpha1)) + exp(alpha1) + alpha1 * exp(alpha1) - exp(alpha1) * log(s2) - (exp(alpha1) * s1 / s2))
    p_beta10 = sum(nij1.sum.j - (s1 * s3 / s2))
    p_beta11 = sum(z1 * nij1.sum.j - z1 * (s1 * s3 / s2))
    p_beta12 = sum(z2 * nij1.sum.j - z2 * (s1 * s3 / s2))
    p_dl11 = sum(ni11 - (s1 * linear_comp_exp * dl11 / s2))
    p_dl21 = sum(ni21 - (s1 * linear_comp_exp * dl21 / s2))
    p_dl31 = sum(ni31 - (s1 * linear_comp_exp * dl31 / s2))
    p_dl41 = sum(ni41 - (s1 * linear_comp_exp * dl41 / s2))
    p_dl51 = sum(ni51 - (s1 * linear_comp_exp * dl51 / s2))

    
    c(p_alpha1, p_beta10, p_beta11, p_beta12, p_dl11, p_dl21, p_dl31, p_dl41, p_dl51)
}

In [18]:
options(warn = -1)
dt = generate.data(N = 300, alpha = c(3, 2), beta1 = c(0.5, 0.5, -0.5), beta2 = c(0.5, -0.5, 0.5), omega = 0.5)
s = c(log(3)+runif(1, -0.05, 0.05), 0.5+runif(1, -0.05, 0.05), 0.5+runif(1, -0.05, 0.05),
      -0.5+runif(1, -0.05, 0.05), log(0.05)+runif(1, -0.01, 0.01), log(0.05)+runif(1, -0.01, 0.01),
      log(0.05)+runif(1, -0.01, 0.01), log(0.05)+runif(1, -0.01, 0.01), log(0.05)+runif(1, -0.01, 0.01))
p = unlist(optim(par = s, fn = LL1.new, data = dt, gr = gra1.new, method = "BFGS")$par)
p[1] = exp(p[1])
p[5] = exp(p[5])
p[6] = exp(p[6])
p[7] = exp(p[7])
p[8] = exp(p[8])
p[9] = exp(p[9])
print(p)

[1]  3.03543568  0.53769727  0.47395119 -0.50950430  0.05006130  0.04977537
[7]  0.05045367  0.05010296  0.05019942


$$
LL2 = \sum^m_{i=1}\left[\sum^n_{j=1}n_{ij}^{(2)}\Lambda_{j2}+log(\Gamma(e^{\alpha_2} + \sum^n_{j=1}n_{ij}^{(2)})) - log(\Gamma(e^{\alpha_2})) - log(\Gamma(\sum^n_{j=1}n_{ij}^{(2)} + 1)) + \alpha_2 e^{\alpha_2} + (\beta_2' Z)\sum^n_{j=1}n_{ij}^{(2)} - (e^{\alpha_2} + \sum^n_{j=1}n_{ij}^{(2)})log(e^{\alpha_2} + e^{\beta_2' Z}\sum^n_{j=1}e^{\Lambda_{j2}})\right]
$$

In [19]:
LL2.new = function(par, data){
    # par = c(alpha2, beta20, beta21, beta22, dl12, dl22, dl32, dl42, dl52)

    alpha2 = par[1]
    beta20 = par[2]
    beta21 = par[3]
    beta22 = par[4]
    dl12 = par[5]
    dl22 = par[6]
    dl32 = par[7]
    dl42 = par[8]
    dl52 = par[9]
    
    ni12 = data[, 6]
    ni22 = data[, 7]
    ni32 = data[, 8]
    ni42 = data[, 9]
    ni52 = data[, 10]
    z1 = data[, 11]
    z2 = data[, 12]

    nij2.sum.j = ni12 + ni22 + ni32 + ni42 + ni52
    dlj2.sum.j = exp(dl12) + exp(dl22) + exp(dl32) + exp(dl42) + exp(dl52)
    
    
    C1 = sum(ni12 * dl12 + ni22 * dl22 + ni32 * dl32 + ni42 * dl42 + ni52 * dl52)
    C2 = log(gamma(exp(alpha2) + nij2.sum.j))
    C3 = log(gamma(exp(alpha2)))
    C4 = log(gamma(nij2.sum.j + 1))
    C5 = alpha2 * exp(alpha2)
    C6 = (beta20 + beta21 * z1 + beta22 * z2) * nij2.sum.j
    C7 = (exp(alpha2) + nij2.sum.j) * log(exp(alpha2) + exp(beta20 + beta21 * z1 + beta22 * z2) * dlj2.sum.j)

    return ((-1) * sum(C1 + C2 - C3 - C4 + C5 + C6 - C7))
}

$$
\frac{\partial}{\partial \alpha_2}LL2 = \sum_{i=1}^m\left\{\frac{[\Gamma(e^{\alpha_2} + \sum^n_{j=1}n_{ij}^{(2)})]'}{\Gamma(e^{\alpha_2} + \sum^n_{j=1}n_{ij}^{(2)})} - \frac{[\Gamma(e^{\alpha_2})]'}{\Gamma(e^{\alpha_2})} + e^{\alpha_2} + \alpha_2 e^{\alpha2} - e^{\alpha_2}log(e^{\alpha_2} + e^{\beta_2' Z}\sum^n_{j=1}e^{\Lambda_{j2}}) - \frac{e^{\alpha_2}(e^{\alpha_2} + \sum^n_{j=1}n_{ij}^{(2)})}{\alpha_2 + e^{\beta_2' Z}\sum^n_{j=1}\Lambda_{j2}}\right\}
$$

$$
\frac{\partial}{\partial \beta_{20}} LL2 = \sum^m_{i=1}\left\{\sum^n_{j=1}n_{ij}^{(2)} - (e^{\alpha_2} + \sum^n_{j=1}n_{ij}^{(2)})\frac{e^{\beta_2'Z}\sum^n_{j=1}e^{\Lambda_{j2}}}{e^{\alpha_2} + e^{\beta_2'Z}\sum^n_{j=1}e^{\Lambda_{j2}}}\right\}
$$

$$
\frac{\partial}{\partial \beta_{21}} LL2 = \sum^m_{i=1}\left\{z_2\sum^n_{j=1}n_{ij}^{(2)} - (e^{\alpha_2} + \sum^n_{j=1}n_{ij}^{(2)})\frac{z_2 e^{\beta_2'Z}\sum^n_{j=1}e^{\Lambda_{j2}}}{e^{\alpha_2} + e^{\beta_2'Z}\sum^n_{j=1}e^{\Lambda_{j2}}}\right\}
$$

$$
\frac{\partial}{\partial \beta_{22}} LL2 = \sum^m_{i=1}\left\{z_2\sum^n_{j=1}n_{ij}^{(2)} - (e^{\alpha_2} + \sum^n_{j=1}n_{ij}^{(2)})\frac{z_2 e^{\beta_2'Z}\sum^n_{j=1}e^{\Lambda_{j2}}}{e^{\alpha_2} + e^{\beta_2'Z}\sum^n_{j=1}e^{\Lambda_{j2}}}\right\}
$$

$$
\frac{\partial}{\partial \Lambda_{j2}} LL2 = \sum^m_{i=1}\left\{n_{ij}^{(2)} - (e^{\alpha_2} + \sum^n_{j=1}n_{ij}^{(2)})\frac{e^{\beta_2'Z }e^{\Lambda_{j2}}}{e^{\alpha_2} + e^{\beta_2' Z}\sum^n_{j=1}e^{\Lambda_{j2}}} \right\}
$$
where j = 1, 2, 3, 4, 5

In [20]:
gra2.new = function(par, data){
    # par = c(alpha2, beta20, beta21, beta22, dl12, dl22, dl32, dl42, dl52)
    
    alpha2 = par[1]
    beta20 = par[2]
    beta21 = par[3]
    beta22 = par[4]
    dl12 = par[5]
    dl22 = par[6]
    dl32 = par[7]
    dl42 = par[8]
    dl52 = par[9]
    
    ni12 = data[, 6]
    ni22 = data[, 7]
    ni32 = data[, 8]
    ni42 = data[, 9]
    ni52 = data[, 10]
    z1 = data[, 11]
    z2 = data[, 12]

    nij2.sum.j = ni12 + ni22 + ni32 + ni42 + ni52
    dlj2.sum.j = exp(dl12) + exp(dl22) + exp(dl32) + exp(dl42) + exp(dl52)

    linear_comp = beta20 + beta21 * z1 + beta22 * z2
    linear_comp_exp = exp(linear_comp)
    s1 = exp(alpha2) + nij2.sum.j
    s2 = exp(alpha2) + linear_comp_exp * dlj2.sum.j
    s3 = linear_comp_exp * dlj2.sum.j

    p_alpha2 = sum(digamma(exp(alpha2) + nij2.sum.j) - digamma(exp(alpha2)) + exp(alpha2) + alpha2 * exp(alpha2) - exp(alpha2) * log(s2) - (exp(alpha2) * s1 / s2))
    p_beta20 = sum(nij2.sum.j - (s1 * s3 / s2))
    p_beta21 = sum(z1 * nij2.sum.j - z1 * (s1 * s3 / s2))
    p_beta22 = sum(z2 * nij2.sum.j - z2 * (s1 * s3 / s2))
    p_dl12 = sum(ni12 - (s1 * linear_comp_exp * dl12 / s2))
    p_dl22 = sum(ni22 - (s1 * linear_comp_exp * dl22 / s2))
    p_dl32 = sum(ni32 - (s1 * linear_comp_exp * dl32 / s2))
    p_dl42 = sum(ni42 - (s1 * linear_comp_exp * dl42 / s2))
    p_dl52 = sum(ni52 - (s1 * linear_comp_exp * dl52 / s2))
    
    c(p_alpha2, p_beta20, p_beta21, p_beta22, p_dl12, p_dl22, p_dl32, p_dl42, p_dl52)
}

In [21]:
options(warn = -1)
dt = generate.data(N=300, alpha = c(3, 2), beta1 = c(0.5, 0.5, -0.5), beta2 = c(0.5, -0.5, 0.5), omega = 0.5)
s = c(log(2)+runif(1, -0.05, 0.05), 0.5+runif(1, -0.05, 0.05), -0.5+runif(1, -0.05, 0.05),
      0.5+runif(1, -0.05, 0.05), log(0.025)+runif(1, -0.01, 0.01), log(0.075)+runif(1, -0.01, 0.01),
      log(0.125)+runif(1, -0.01, 0.01), log(0.175)+runif(1, -0.01, 0.01), log(0.225)+runif(1, -0.01, 0.01))
p = unlist(optim(par = s, fn = LL2.new, data = dt, gr = gra2.new, method = "BFGS")$par)
p[1] = exp(p[1])
p[5] = exp(p[5])
p[6] = exp(p[6])
p[7] = exp(p[7])
p[8] = exp(p[8])
p[9] = exp(p[9])
print(p)

[1]  2.01913150  0.51643468 -0.46616058  0.54124349  0.02510965  0.07528875
[7]  0.12536268  0.17351959  0.22418491


In [22]:
LL.all = function(par, known, data){
    # par = c(alpha1, beta10, beta11, beta12, dl11, dl21, dl31, dl41, dl51, alpha2, beta20, beta21, beta22, dl11, dl21, dl31, dl41, dl51)
    omega = par[1]
    alpha1 = known[1]
    beta10 = known[2]
    beta11 = known[3]
    beta12 = known[4]
    dl11 = known[5]
    dl21 = known[6]
    dl31 = known[7]
    dl41 = known[8]
    dl51 = known[9]
    alpha2 = known[10]
    beta20 = known[11]
    beta21 = known[12]
    beta22 = known[13]
    dl12 = known[14]
    dl22 = known[15]
    dl32 = known[16]
    dl42 = known[17]
    dl52 = known[18]
    ni11 = data[, 1]
    ni21 = data[, 2]
    ni31 = data[, 3]
    ni41 = data[, 4]
    ni51 = data[, 5]
    ni12 = data[, 6]
    ni22 = data[, 7]
    ni32 = data[, 8]
    ni42 = data[, 9]
    ni52 = data[, 10]
    z1 = data[, 11]
    z2 = data[, 12]

    nij1.sum.j = ni11 + ni21 + ni31 + ni41 + ni51
    dlj1.sum.j = dl11 + dl21 + dl31 + dl41 + dl51
    nij2.sum.j = ni12 + ni22 + ni32 + ni42 + ni52
    dlj2.sum.j = dl12 + dl22 + dl32 + dl42 + dl52
    linear1 = beta10 + beta11 * z1 + beta12 * z2
    linear2 = beta20 + beta21 * z1 + beta22 * z2
    exp1 = exp(beta10 + beta11 * z1 + beta12 * z2)
    exp2 = exp(beta20 + beta21 * z1 + beta22 * z2)

    C1 = sum(ni11 * log(dl11) + ni21 * log(dl21) + ni31 * log(dl31) + ni41 * log(dl41) + ni51 * log(dl51))
    C2 = log(gamma(alpha1 + nij1.sum.j))
    C3 = log(gamma(alpha1))
    C4 = log(gamma(nij1.sum.j + 1))
    C5 = alpha1 * log(alpha1)
    C6 = linear1 * nij1.sum.j
    C7 = (alpha1 + nij1.sum.j) * log(alpha1 + exp1 * dlj1.sum.j)

    C8 = sum(ni12 * log(dl12) + ni22 * log(dl22) + ni32 * log(dl32) + ni42 * log(dl42) + ni52 * log(dl52))
    C9 = log(gamma(alpha2 + nij2.sum.j))
    C10 = log(gamma(alpha2))
    C11 = log(gamma(nij2.sum.j + 1))
    C12 = alpha2 * log(alpha2)
    C13 = linear2 * nij2.sum.j
    C14 = (alpha2 + nij2.sum.j) * log(alpha2 + exp2 * dlj2.sum.j)

    C151 = ((alpha1 + exp1 * dlj1.sum.j)/(alpha1 + exp1 * dlj1.sum.j + 1))^(alpha1 + nij1.sum.j)
    C152 = (alpha1/(alpha1 + 1))^(alpha1)
    C153 = ((alpha2 + exp2 * dlj2.sum.j)/(alpha2 + exp2 * dlj2.sum.j + 1))^(alpha2 + nij2.sum.j)
    C154 = (alpha2/(alpha2 + 1))^(alpha2)
    C15 = 1 + omega * (C151 - C152) * (C153 - C154)
    return ((-1)*sum(C1 + C2 - C3 - C4 + C5 + C6 - C7 + C8 + C9 - C10 - C11 + C12 + C13 - C14 + C15))
}

In [23]:
options(warn = -1)
dt = generate.data(N=300, alpha = c(3, 2), beta1 = c(0.5, 0.5, -0.5), beta2 = c(0.5, -0.5, 0.5), omega = 0.5)
s1 = c(log(3)+runif(1, -0.05, 0.05), 0.5+runif(1, -0.05, 0.05), 0.5+runif(1, -0.05, 0.05),
      -0.5+runif(1, -0.05, 0.05), log(0.05)+runif(1, -0.01, 0.01), log(0.05)+runif(1, -0.01, 0.01),
      log(0.05)+runif(1, -0.01, 0.01), log(0.05)+runif(1, -0.01, 0.01), log(0.05)+runif(1, -0.01, 0.01))
p1 = unlist(optim(par = s1, fn = LL1.new, data = dt, gr = gra1.new, method = "BFGS")$par)
p1[c(1, 5, 6, 7, 8, 9)] = exp(p1[c(1, 5, 6, 7, 8, 9)])

s2 = c(log(2)+runif(1, -0.05, 0.05), 0.5+runif(1, -0.05, 0.05), -0.5+runif(1, -0.05, 0.05),
      0.5+runif(1, -0.05, 0.05), log(0.025)+runif(1, -0.01, 0.01), log(0.075)+runif(1, -0.01, 0.01),
      log(0.125)+runif(1, -0.01, 0.01), log(0.175)+runif(1, -0.01, 0.01), log(0.225)+runif(1, -0.01, 0.01))
p2 = unlist(optim(par = s2, fn = LL2.new, data = dt, gr = gra2.new, method = "BFGS")$par)
p2[c(1, 5, 6, 7, 8, 9)] = exp(p2[c(1, 5, 6, 7, 8, 9)])

p = c(p1, p2)

ome = optimize(f = LL.all, interval = c(0.5+runif(1, 0, 0.1), 0.5+runif(1, -0.1, 0)), known = p, data = dt)$minimum
p = c(p, ome)
print(p)

 [1]  2.92015108  0.53081203  0.52291535 -0.45072505  0.05036257  0.05017778
 [7]  0.04961119  0.04964565  0.05036580  2.05931683  0.47753809 -0.49005175
[13]  0.45874772  0.02498700  0.07501246  0.12405844  0.17468204  0.22694611
[19]  0.57873170


In [24]:
options(warn = -1)
param.new = c()
for (i in 1:1000){
    dt = generate.data(N=300, alpha = c(3, 2), beta1 = c(0.5, 0.5, -0.5), beta2 = c(0.5, -0.5, 0.5), omega = 0.5)
    s1 = c(log(3)+runif(1, -0.05, 0.05), 0.5+runif(1, -0.05, 0.05), 0.5+runif(1, -0.05, 0.05),
        -0.5+runif(1, -0.05, 0.05), log(0.05)+runif(1, -0.01, 0.01), log(0.05)+runif(1, -0.01, 0.01),
        log(0.05)+runif(1, -0.01, 0.01), log(0.05)+runif(1, -0.01, 0.01), log(0.05)+runif(1, -0.01, 0.01))
    p1 = unlist(optim(par = s1, fn = LL1.new, data = dt, gr = gra1.new, method = "BFGS")$par)
    p1[c(1, 5, 6, 7, 8, 9)] = exp(p1[c(1, 5, 6, 7, 8, 9)])

    s2 = c(log(2)+runif(1, -0.05, 0.05), 0.5+runif(1, -0.05, 0.05), -0.5+runif(1, -0.05, 0.05),
        0.5+runif(1, -0.05, 0.05), log(0.025)+runif(1, -0.01, 0.01), log(0.075)+runif(1, -0.01, 0.01),
        log(0.125)+runif(1, -0.01, 0.01), log(0.175)+runif(1, -0.01, 0.01), log(0.225)+runif(1, -0.01, 0.01))
    p2 = unlist(optim(par = s2, fn = LL2.new, data = dt, gr = gra2.new, method = "BFGS")$par)
    p2[c(1, 5, 6, 7, 8, 9)] = exp(p2[c(1, 5, 6, 7, 8, 9)])
    p = c(p1, p2)
    ome = optimize(f = LL.all, interval = c(0.5+runif(1, 0, 0.1), 0.5+runif(1, -0.1, 0)), known = p, data = dt)$minimum
    p = c(p, ome)
    param.new = rbind(param.new, p) 
}
colnames(param.new) = c("alpha1", "beta10", "beta11", "beta12", "dlambda11", "dlambda21", "dlambda31", "dlambda41", "dlambda51",
                        "alpha2", "beta20", "beta21", "beta22", "dlambda12", "dlambda22", "dlambda32", "dlambda42", "dlambda52",
                        "omega")
print(param.new)

    alpha1    beta10    beta11     beta12  dlambda11  dlambda21  dlambda31
p 2.890233 0.5090879 0.4791436 -0.4680189 0.05047933 0.05005266 0.04955187
p 2.909185 0.5398490 0.4747812 -0.4502546 0.04983173 0.05003054 0.04974264
p 3.061099 0.4966211 0.4527105 -0.5457237 0.05039467 0.04998571 0.04954060
p 3.081581 0.4514155 0.4635393 -0.4924398 0.04976031 0.05034649 0.05036059
p 2.979614 0.4852239 0.5380680 -0.5467007 0.05034962 0.05005454 0.05030530
p 3.085510 0.4700438 0.5145404 -0.4782790 0.05020061 0.04984827 0.04959366
p 3.027334 0.5055210 0.5124229 -0.5282605 0.04956636 0.05041179 0.04987947
p 2.870269 0.5126097 0.5217938 -0.4689657 0.05034568 0.04987355 0.05003779
p 2.993372 0.4880912 0.5207179 -0.5126703 0.05047097 0.05027980 0.05043684
p 2.948702 0.4987610 0.4600449 -0.5281798 0.05015475 0.05026919 0.04962214
p 2.880410 0.4835028 0.5223880 -0.4772530 0.05023416 0.05009351 0.05016973
p 3.009624 0.4510193 0.5188475 -0.4908137 0.05038406 0.04964818 0.05021331
p 2.975695 0.4830383 0.46

In [25]:
# mean
print(round(apply(param.new, 2, mean), 4))
cat("===============================================================================\n")
# bias
print(round(apply(param.new, 2, mean) - c(3, 0.5, 0.5, -0.5, 0.05, 0.05, 0.05, 0.05, 0.05, 2, 0.5, -0.5, 0.5, 0.025, 0.075, 0.125, 0.175, 0.225, 0.5), 4))
cat("===============================================================================\n")
# standard deviation
print(round(apply(param.new, 2, sd), 4))

   alpha1    beta10    beta11    beta12 dlambda11 dlambda21 dlambda31 dlambda41 
   3.0006    0.5003    0.5003   -0.5014    0.0500    0.0500    0.0500    0.0500 
dlambda51    alpha2    beta20    beta21    beta22 dlambda12 dlambda22 dlambda32 
   0.0500    2.0010    0.4992   -0.5006    0.5007    0.0250    0.0750    0.1249 
dlambda42 dlambda52     omega 
   0.1750    0.2250    0.5043 
   alpha1    beta10    beta11    beta12 dlambda11 dlambda21 dlambda31 dlambda41 
   0.0006    0.0003    0.0003   -0.0014    0.0000    0.0000    0.0000    0.0000 
dlambda51    alpha2    beta20    beta21    beta22 dlambda12 dlambda22 dlambda32 
   0.0000    0.0010   -0.0008   -0.0006    0.0007    0.0000    0.0000   -0.0001 
dlambda42 dlambda52     omega 
   0.0000    0.0000    0.0043 
   alpha1    beta10    beta11    beta12 dlambda11 dlambda21 dlambda31 dlambda41 
   0.0860    0.0280    0.0288    0.0294    0.0003    0.0003    0.0003    0.0003 
dlambda51    alpha2    beta20    beta21    beta22 dlambda12 dlambd

---

#### Calculate Fisher

$$
logL = \sum^m_{i=1}\left\{\sum^n_{j=1}n_{ij}^{(1)}\Lambda_{j1}+log(\Gamma(e^{\alpha_1} + \sum^n_{j=1}n_{ij}^{(1)})) - log(\Gamma(e^{\alpha_1})) - log(\Gamma(\sum^n_{j=1}n_{ij}^{(1)} + 1)) + \alpha_1 e^{\alpha_1} + (\beta_1' Z)\sum^n_{j=1}n_{ij}^{(1)} - (e^{\alpha_1} + \sum^n_{j=1}n_{ij}^{(1)})log(e^{\alpha_1} + e^{\beta_1' Z}\sum^n_{j=1}e^{\Lambda_{j1}}) + \sum^n_{j=1}n_{ij}^{(2)}\Lambda_{j2}+log(\Gamma(e^{\alpha_2} + \sum^n_{j=1}n_{ij}^{(2)})) - log(\Gamma(e^{\alpha_2})) - log(\Gamma(\sum^n_{j=1}n_{ij}^{(2)} + 1)) + \alpha_2 e^{\alpha_2} + (\beta_2' Z)\sum^n_{j=1}n_{ij}^{(2)} - (e^{\alpha_2} + \sum^n_{j=1}n_{ij}^{(2)})log(e^{\alpha_2} + e^{\beta_2' Z}\sum^n_{j=1}e^{\Lambda_{j2}})+log\left[1 + \omega\left(\left(\frac{e^{\alpha_1} + e^{\beta_1'Z}\sum^n_{j=1}e^{\Lambda_{j1}}}{e^{\alpha_1} + e^{\beta_1'Z}\sum^n_{j=1}e^{\Lambda_{j1}}+1}\right)^{e^{\alpha_1} + \sum^n_{j=1}n_{ij}^{(1)}} - \left(\frac{e^{\alpha_1}}{e^{\alpha_1} + 1}\right)^{e^{\alpha_1}}\right) \left(\left(\frac{e^{\alpha_2} + e^{\beta_2'Z}\sum^n_{j=1}e^{\Lambda_{j2}}}{e^{\alpha_2} + e^{\beta_2'Z}\sum^n_{j=1}e^{\Lambda_{j2}}+1}\right)^{e^{\alpha_2} + \sum^n_{j=1}n_{ij}^{(2)}} - \left(\frac{e^{\alpha_2}}{e^{\alpha_2} + 1}\right)^{e^{\alpha_2}}\right) \right]\right\}
$$

In [26]:
logL.new = function(par, data){
    alpha1 = par[1]
    beta10 = par[2]
    beta11 = par[3]
    beta12 = par[4]
    dl11 = par[5]
    dl21 = par[6]
    dl31 = par[7]
    dl41 = par[8]
    dl51 = par[9]
    alpha2 = par[10]
    beta20 = par[11]
    beta21 = par[12]
    beta22 = par[13]
    dl12 = par[14]
    dl22 = par[15]
    dl32 = par[16]
    dl42 = par[17]
    dl52 = par[18]
    omega = par[19]

    ni11 = data[, 1]
    ni21 = data[, 2]
    ni31 = data[, 3]
    ni41 = data[, 4]
    ni51 = data[, 5]
    ni12 = data[, 6]
    ni22 = data[, 7]
    ni32 = data[, 8]
    ni42 = data[, 9]
    ni52 = data[, 10]
    z1 = data[, 11]
    z2 = data[, 12]

    nij1.sum.j = ni11 + ni21 + ni31 + ni41 + ni51
    dlj1.sum.j = exp(dl11) + exp(dl21) + exp(dl31) + exp(dl41) + exp(dl51)
    
    nij2.sum.j = ni12 + ni22 + ni32 + ni42 + ni52
    dlj2.sum.j = exp(dl12) + exp(dl22) + exp(dl32) + exp(dl42) + exp(dl52)

    exp1 = exp(beta10 + beta11 * z1 + beta12 * z2)
    exp2 = exp(beta20 + beta21 * z1 + beta22 * z2)

    C1 = sum(ni11 * dl11 + ni21 * dl21 + ni31 * dl31 + ni41 * dl41 + ni51 * dl51)
    C2 = log(gamma(exp(alpha1) + nij1.sum.j))
    C3 = log(gamma(exp(alpha1)))
    C4 = log(gamma(nij1.sum.j + 1))
    C5 = alpha1 * exp(alpha1)
    C6 = (beta10 + beta11 * z1 + beta12 * z2) * nij1.sum.j
    C7 = (exp(alpha1) + nij1.sum.j) * log(exp(alpha1) + exp(beta10 + beta11 * z1 + beta12 * z2) * dlj1.sum.j)

    C8 = sum(ni12 * dl12 + ni22 * dl22 + ni32 * dl32 + ni42 * dl42 + ni52 * dl52)
    C9 = log(gamma(exp(alpha2) + nij2.sum.j))
    C10 = log(gamma(exp(alpha2)))
    C11 = log(gamma(nij2.sum.j + 1))
    C12 = alpha2 * exp(alpha2)
    C13 = (beta20 + beta21 * z1 + beta22 * z2) * nij2.sum.j
    C14 = (exp(alpha2) + nij2.sum.j) * log(exp(alpha2) + exp(beta20 + beta21 * z1 + beta22 * z2) * dlj2.sum.j)

    C151 = ((exp(alpha1) + exp1 * dlj1.sum.j)/(exp(alpha1) + exp1 * dlj1.sum.j + 1))^(exp(alpha1) + nij1.sum.j)
    C152 = (exp(alpha1)/(exp(alpha1) + 1))^(exp(alpha1))
    C153 = ((exp(alpha2) + exp2 * dlj2.sum.j)/(exp(alpha2) + exp2 * dlj2.sum.j + 1))^(exp(alpha2) + nij2.sum.j)
    C154 = (exp(alpha2)/(exp(alpha2) + 1))^(exp(alpha2))
    C15 = 1 + omega * (C151 - C152) * (C153 - C154)

    return ((-1)*sum(C1 + C2 - C3 - C4 + C5 + C6 - C7 + C8 + C9 - C10 - C11 + C12 + C13 - C14 + C15))
}

$$
s11 = e^{\alpha_1} + e^{\beta_1'Z}\sum^n_{j=1}e^{\Lambda_{j1}}
$$

$$
s12 = e^{\alpha_1} + \sum^n_{j=1}n_{ij}^{(1)}
$$

$$
s21 = e^{\alpha_2} + e^{\beta_2'Z}\sum^n_{j=1}e^{\Lambda_{j2}}
$$

$$
s22 = e^{\alpha_2} + \sum^n_{j=1}n_{ij}^{(2)}
$$

$$
c1 = (\frac{s11}{s11 + 1})^{s12} - (\frac{e^{\alpha_1}}{e^{\alpha_1} + 1})^{e^{\alpha_1}}
$$

$$
c2 = (\frac{s21}{s21 + 1})^{s22} - (\frac{e^{\alpha_2}}{e^{\alpha_2} + 1})^{e^{\alpha_2}}
$$

$$
denominator = 1 + \omega ((\frac{s11}{s11 + 1})^{s12} - (\frac{\alpha_1}{\alpha_1 + 1})^{\alpha_1})((\frac{s21}{s21 + 1})^{s22} - (\frac{\alpha_2}{\alpha_2 + 1})^{\alpha_2}) = 1 + \omega\times c1 \times c2
$$

$$
\frac{\partial}{\partial \alpha_1}logL = \sum_{i=1}^m\left\{\frac{[\Gamma(e^{\alpha_1} + \sum^n_{j=1}n_{ij}^{(1)})]'}{\Gamma(e^{\alpha_1} + \sum^n_{j=1}n_{ij}^{(1)})} - \frac{[\Gamma(e^{\alpha_1})]'}{\Gamma(e^{\alpha_1})} + e^{\alpha_1} + \alpha_1 e^{\alpha1} - e^{\alpha_1}log(e^{\alpha_1} + e^{\beta_1' Z}\sum^n_{j=1}e^{\Lambda_{j1}}) - \frac{e^{\alpha_1}(e^{\alpha_1} + \sum^n_{j=1}n_{ij}^{(1)})}{\alpha_1 + e^{\beta_1' Z}\sum^n_{j=1}\Lambda_{j1}} + \frac{\omega \times c2 \times [c1(e^{\alpha_1}log(\frac{s11}{s11 + 1}) + s12\times (\frac{e^{\alpha_1}}{s11} - \frac{e^{\alpha_1}}{s11 + 1})) - (\frac{e^{\alpha_1}}{e^{\alpha_1} + 1})^{e^{\alpha_1}}(e^{\alpha_1}log(\frac{e^{\alpha_1}}{e^{\alpha_1} + 1})) + e^{\alpha_1}(1 - \frac{e^{\alpha_1}}{e^{\alpha_1} + 1})]}{denominator}\right\}
$$

$$
\frac{\partial}{\partial \beta_{10}} logL = \sum^m_{i=1}\left\{\sum^n_{j=1}n_{ij}^{(1)} - (e^{\alpha_1} + \sum^n_{j=1}n_{ij}^{(1)})\frac{e^{\beta_1'Z}\sum^n_{j=1}e^{\Lambda_{j1}}}{e^{\alpha_1} + e^{\beta_1'Z}\sum^n_{j=1}e^{\Lambda_{j1}}} + \frac{\omega \times c2 \times (\frac{s11}{s11 + 1})^{s12} \times [s12(\frac{e^{\beta_1'Z}\sum\Lambda_{j1}}{s11} - \frac{e^{\beta_1'Z}\sum\Lambda_{j1}}{s11 + 1})]}{denominator}\right\}
$$

$$
\frac{\partial}{\partial \beta_{11}} logL = \sum^m_{i=1}\left\{z_1\sum^n_{j=1}n_{ij}^{(1)} - (e^{\alpha_1} + \sum^n_{j=1}n_{ij}^{(1)})\frac{z_1 e^{\beta_1'Z}\sum^n_{j=1}e^{\Lambda_{j1}}}{e^{\alpha_1} + e^{\beta_1'Z}\sum^n_{j=1}e^{\Lambda_{j1}}} + \frac{\omega \times c2 \times (\frac{s11}{s11 + 1})^{s12} \times [s12(\frac{z_1 e^{\beta_1'Z}\sum\Lambda_{j1}}{s11} - \frac{z_1 e^{\beta_1'Z}\sum\Lambda_{j1}}{s11 + 1})]}{denominator}\right\}
$$

$$
\frac{\partial}{\partial \beta_{12}} logL = \sum^m_{i=1}\left\{z_2\sum^n_{j=1}n_{ij}^{(1)} - (e^{\alpha_1} + \sum^n_{j=1}n_{ij}^{(1)})\frac{z_2 e^{\beta_1'Z}\sum^n_{j=1}e^{\Lambda_{j1}}}{e^{\alpha_1} + e^{\beta_1'Z}\sum^n_{j=1}e^{\Lambda_{j1}}} + \frac{\omega \times c2 \times (\frac{s11}{s11 + 1})^{s12} \times [s12(\frac{z_2 e^{\beta_1'Z}\sum\Lambda_{j1}}{s11} - \frac{z_2 e^{\beta_1'Z}\sum\Lambda_{j1}}{s11 + 1})]}{denominator}\right\}
$$

$$
\frac{\partial}{\partial \Lambda_{j1}} logL = \sum^m_{i=1}\left\{n_{ij}^{(1)} - (e^{\alpha_1} + \sum^n_{j=1}n_{ij}^{(1)})\frac{e^{\beta_1'Z }e^{\Lambda_{j1}}}{e^{\alpha_1} + e^{\beta_1' Z}\sum^n_{j=1}e^{\Lambda_{j1}}} +  \frac{\omega \times c2 \times (\frac{s11}{s11 + 1})^{s12} \times [s12(\frac{e^{\beta_1'Z}e^{\Lambda_{j1}}}{s11} - \frac{e^{\beta_1'Z}e^{\Lambda_{j1}}}{s11 + 1})]}{denominator}\right\}
$$
where j = 1, 2, 3, 4, 5

$$
\frac{\partial}{\partial \alpha_2}logL = \sum_{i=1}^m\left\{\frac{[\Gamma(e^{\alpha_2} + \sum^n_{j=1}n_{ij}^{(2)})]'}{\Gamma(e^{\alpha_2} + \sum^n_{j=1}n_{ij}^{(2)})} - \frac{[\Gamma(e^{\alpha_2})]'}{\Gamma(e^{\alpha_2})} + e^{\alpha_2} + \alpha_2 e^{\alpha_2} - e^{\alpha_2}log(e^{\alpha_2} + e^{\beta_2' Z}\sum^n_{j=1}e^{\Lambda_{j2}}) - \frac{e^{\alpha_2}(\alpha_2 + \sum^n_{j=1}n_{ij}^{(2)})}{\alpha_2 + e^{\beta_2' Z}\sum^n_{j=1}\Lambda_{j2}} + \frac{\omega \times c1 \times [c2(e^{\alpha_2}log(\frac{s21}{s21 + 1}) + c22\times (\frac{e^{\alpha_2}}{s21} - \frac{e^{\alpha_2}}{s21 + 1})) - (\frac{\alpha_2}{\alpha_2 + 1})^{e^{\alpha_2}}(e^{\alpha_2}log(\frac{e^{\alpha_2}}{e^{\alpha_2} + 1})) + e^{\alpha_2}(1 - \frac{e^{\alpha_2}}{e^{\alpha_2 + 1}})]}{denominator}\right\}
$$

$$
\frac{\partial}{\partial \beta_{20}} logL = \sum^m_{i=1}\left\{\sum^n_{j=1}n_{ij}^{(2)} - (e^{\alpha_2} + \sum^n_{j=1}n_{ij}^{(2)})\frac{e^{\beta_2'Z}\sum^n_{j=1}e^{\Lambda_{j2}}}{e^{\alpha_2} + e^{\beta_2'Z}\sum^n_{j=1}e^{\Lambda_{j2}}} + \frac{\omega \times c1 \times (\frac{s21}{s21 + 1})^{s22} \times [s22(\frac{e^{\beta_2'Z}\sum\Lambda_{j2}}{s21} - \frac{e^{\beta_2'Z}\sum\Lambda_{j2}}{s21 + 1})]}{denominator}\right\}
$$

$$
\frac{\partial}{\partial \beta_{21}} logL = \sum^m_{i=1}\left\{z_1\sum^n_{j=1}n_{ij}^{(2)} - (e^{\alpha_2} + \sum^n_{j=1}n_{ij}^{(2)})\frac{z_1 e^{\beta_2'Z}\sum^n_{j=1}e^{\Lambda_{j2}}}{e^{\alpha_2} + e^{\beta_2'Z}\sum^n_{j=1}e^{\Lambda_{j2}}}  \frac{\omega \times c1 \times (\frac{s21}{s21 + 1})^{s22} \times [s22(\frac{z_1 e^{\beta_2'Z}\sum\Lambda_{j2}}{s21} - \frac{z_1 e^{\beta_2'Z}\sum\Lambda_{j2}}{s21 + 1})]}{denominator}\right\}
$$

$$
\frac{\partial}{\partial \beta_{22}} logL = \sum^m_{i=1}\left\{z_2\sum^n_{j=1}n_{ij}^{(2)} - (e^{\alpha_2} + \sum^n_{j=1}n_{ij}^{(2)})\frac{z_2 e^{\beta_2'Z}\sum^n_{j=1}e^{\Lambda_{j1}}}{e^{\alpha_1} + e^{\beta_1'Z}\sum^n_{j=1}e^{\Lambda_{j2}}}  \frac{\omega \times c1 \times (\frac{s21}{s21 + 1})^{s22} \times [s22(\frac{z_2 e^{\beta_2'Z}\sum\Lambda_{j2}}{s21} - \frac{z_2 e^{\beta_2'Z}\sum\Lambda_{j2}}{s21 + 1})]}{denominator}\right\}
$$

$$
\frac{\partial}{\partial \Lambda_{j2}} logL = \sum^m_{i=1}\left\{n_{ij}^{(2)} - (e^{\alpha_2} + \sum^n_{j=1}n_{ij}^{(2)})\frac{e^{\beta_2'Z }e^{\Lambda_{j2}}}{e^{\alpha_2} + e^{\beta_2' Z}\sum^n_{j=1}e^{\Lambda_{j2}}} +  \frac{\omega \times c1 \times (\frac{s21}{s21 + 1})^{s22} \times [s22(\frac{e^{\beta_2'Z}e^{\Lambda_{j2}}}{s21} - \frac{e^{\beta_2'Z}e^{\Lambda_{j2}}}{s21 + 1})]}{denominator}\right\}
$$
where j = 1, 2, 3, 4, 5

In [96]:
g = function(par, data){
    
    alpha1 = par[1]
    beta10 = par[2]
    beta11 = par[3]
    beta12 = par[4]
    dl11 = par[5]
    dl21 = par[6]
    dl31 = par[7]
    dl41 = par[8]
    dl51 = par[9]
    alpha2 = par[10]
    beta20 = par[11]
    beta21 = par[12]
    beta22 = par[13]
    dl12 = par[14]
    dl22 = par[15]
    dl32 = par[16]
    dl42 = par[17]
    dl52 = par[18]
    omega = par[19]

    ni11 = data[, 1]
    ni21 = data[, 2]
    ni31 = data[, 3]
    ni41 = data[, 4]
    ni51 = data[, 5]
    ni12 = data[, 6]
    ni22 = data[, 7]
    ni32 = data[, 8]
    ni42 = data[, 9]
    ni52 = data[, 10]
    z1 = data[, 11]
    z2 = data[, 12]

    alpha1_exp = exp(alpha1)
    alpha2_exp = exp(alpha2)
    linear1 = beta10 + beta11 * z1 + beta12 * z2
    linear2 = beta20 + beta21 * z1 + beta22 * z2
    linear1_exp = exp(linear1)
    linear2_exp = exp(linear2)
    nij1.sum.j = ni11 + ni21 + ni31 + ni41 + ni51
    nij2.sum.j = ni12 + ni22 + ni32 + ni42 + ni52
    dlj1.sum.j = exp(dl11) + exp(dl21) + exp(dl31) + exp(dl41) + exp(dl51)
    dlj2.sum.j = exp(dl12) + exp(dl22) + exp(dl32) + exp(dl42) + exp(dl52)
    s11 = alpha1_exp + linear1_exp * dlj1.sum.j
    s12 = alpha1_exp + nij1.sum.j
    s21 = alpha2_exp + linear2_exp * dlj2.sum.j
    s22 = alpha2_exp + nij2.sum.j
    c1 = (s11/(s11+1))^(s12) - (alpha1_exp/(alpha1_exp+1))^(alpha1_exp)
    c2 = (s21/(s21+1))^(s22) - (alpha2_exp/(alpha2_exp+1))^(alpha2_exp)
    denominator = 1 + omega * c1 * c2

    p_alpha1 = sum(digamma(s12) - digamma(alpha1_exp) + alpha1_exp + alpha1 * alpha1_exp - alpha1_exp * log(s11) - alpha1_exp * s12 / s11 + 
                   omega * c2 * ( c1 * (alpha1_exp * log(s11/(s11+1)) + s12 * (alpha1_exp/s11 - alpha1_exp/(s11+1))) - (alpha1_exp/(alpha1_exp))^(alpha1_exp) *
                   alpha1_exp * (1 - alpha1_exp/(alpha1_exp+1) + log(alpha1_exp/(alpha1_exp+1))) ) / denominator)
    p_beta10 = sum(nij1.sum.j - s12 * linear1_exp * dlj1.sum.j / s11 + omega * c2 * (s11/(s11+1))^(s12) * s12 * (linear1_exp*dlj1.sum.j/s11 - linear1_exp*dlj1.sum.j/(s11+1)) / denominator)
    p_beta11 = sum(z1 * nij1.sum.j - s12 * z1 * linear1_exp * dlj1.sum.j / s11 + omega * c2 * (s11/(s11+1))^(s12) * s12 * (z1*linear1_exp*dlj1.sum.j/s11 - z1*linear1_exp*dlj1.sum.j/(s11+1)) / denominator)
    p_beta12 = sum(z2 * nij1.sum.j - s12 * z2 * linear1_exp * dlj1.sum.j / s11 + omega * c2 * (s11/(s11+1))^(s12) * s12 * (z2*linear1_exp*dlj1.sum.j/s11 - z2*linear1_exp*dlj1.sum.j/(s11+1)) / denominator)
    p_dl11 = sum(ni11 - s12 * linear1_exp * dl11/s11 + omega * c2 * (s11/(s11+1))^(s12) * s12 * (linear1_exp*dl11/s11 - linear1_exp*dl11/(s11+1)) / denominator)
    p_dl21 = sum(ni21 - s12 * linear1_exp * dl21/s11 + omega * c2 * (s11/(s11+1))^(s12) * s12 * (linear1_exp*dl21/s11 - linear1_exp*dl21/(s11+1)) / denominator)
    p_dl31 = sum(ni31 - s12 * linear1_exp * dl31/s11 + omega * c2 * (s11/(s11+1))^(s12) * s12 * (linear1_exp*dl31/s11 - linear1_exp*dl31/(s11+1)) / denominator)
    p_dl41 = sum(ni41 - s12 * linear1_exp * dl41/s11 + omega * c2 * (s11/(s11+1))^(s12) * s12 * (linear1_exp*dl41/s11 - linear1_exp*dl41/(s11+1)) / denominator)
    p_dl51 = sum(ni51 - s12 * linear1_exp * dl51/s11 + omega * c2 * (s11/(s11+1))^(s12) * s12 * (linear1_exp*dl51/s11 - linear1_exp*dl51/(s11+1)) / denominator)

    p_alpha2 = sum(digamma(s22) - digamma(alpha2_exp) + alpha2_exp + alpha2 * alpha2_exp - alpha2_exp * log(s21) - alpha2_exp * s22 / s21 + 
                   omega * c1 * ( c2 * (alpha2_exp * log(s21/(s21+1)) + s22 * (alpha2_exp/s21 - alpha2_exp/(s21+1))) - (alpha2_exp/(alpha2_exp))^(alpha2_exp) *
                   alpha2_exp * (1 - alpha2_exp/(alpha2_exp+1) + log(alpha2_exp/(alpha2_exp+1))) ) / denominator)
    p_beta20 = sum(nij2.sum.j - s22 * linear2_exp * dlj2.sum.j / s21 + omega * c1 * (s21/(s21+1))^(s22) * s22 * (linear2_exp*dlj2.sum.j/s21 - linear2_exp*dlj2.sum.j/(s21+1)) / denominator)
    p_beta21 = sum(z1 * nij2.sum.j - s22 * z1 * linear2_exp * dlj2.sum.j / s21 + omega * c1 * (s11/(s21+1))^(s22) * s22 * (z1*linear2_exp*dlj2.sum.j/s21 - z1*linear2_exp*dlj1.sum.j/(s21+1)) / denominator)
    p_beta22 = sum(z2 * nij2.sum.j - s22 * z2 * linear2_exp * dlj2.sum.j / s21 + omega * c1 * (s11/(s21+1))^(s22) * s22 * (z2*linear2_exp*dlj2.sum.j/s21 - z2*linear2_exp*dlj1.sum.j/(s21+1)) / denominator)
    p_dl12 = sum(ni12 - s22 * linear2_exp * dl12/s21 + omega * c1 * (s21/(s21+1))^(s22) * s22 * (linear2_exp*dl12/s21 - linear2_exp*dl12/(s21+1)) / denominator)
    p_dl22 = sum(ni22 - s22 * linear2_exp * dl22/s21 + omega * c1 * (s21/(s21+1))^(s22) * s22 * (linear2_exp*dl22/s21 - linear2_exp*dl22/(s21+1)) / denominator)
    p_dl32 = sum(ni32 - s22 * linear2_exp * dl32/s21 + omega * c1 * (s21/(s21+1))^(s22) * s22 * (linear2_exp*dl32/s21 - linear2_exp*dl32/(s21+1)) / denominator)
    p_dl42 = sum(ni42 - s22 * linear2_exp * dl42/s21 + omega * c1 * (s21/(s21+1))^(s22) * s22 * (linear2_exp*dl42/s21 - linear2_exp*dl42/(s21+1)) / denominator)
    p_dl52 = sum(ni52 - s22 * linear2_exp * dl52/s21 + omega * c1 * (s21/(s21+1))^(s22) * s22 * (linear2_exp*dl52/s21 - linear2_exp*dl52/(s21+1)) / denominator)

    p_omega = sum(c1 * c2 / denominator)

    return (c(p_alpha1, p_beta10, p_beta11, p_beta12, p_dl11, p_dl21, p_dl31, p_dl41, p_dl51, p_alpha2, p_beta20, p_beta21, p_beta22, p_dl12, p_dl22, p_dl32, p_dl42, p_dl52, p_omega))
}

In [28]:
p_alpha1 = function(par, data){
    alpha1 = par[1]
    beta10 = par[2]
    beta11 = par[3]
    beta12 = par[4]
    dl11 = par[5]
    dl21 = par[6]
    dl31 = par[7]
    dl41 = par[8]
    dl51 = par[9]
    alpha2 = par[10]
    beta20 = par[11]
    beta21 = par[12]
    beta22 = par[13]
    dl12 = par[14]
    dl22 = par[15]
    dl32 = par[16]
    dl42 = par[17]
    dl52 = par[18]
    omega = par[19]

    ni11 = data[, 1]
    ni21 = data[, 2]
    ni31 = data[, 3]
    ni41 = data[, 4]
    ni51 = data[, 5]
    ni12 = data[, 6]
    ni22 = data[, 7]
    ni32 = data[, 8]
    ni42 = data[, 9]
    ni52 = data[, 10]
    z1 = data[, 11]
    z2 = data[, 12]

    alpha1_exp = exp(alpha1)
    alpha2_exp = exp(alpha2)
    linear1 = beta10 + beta11 * z1 + beta12 * z2
    linear2 = beta20 + beta21 * z1 + beta22 * z2
    linear1_exp = exp(linear1)
    linear2_exp = exp(linear2)
    nij1.sum.j = ni11 + ni21 + ni31 + ni41 + ni51
    nij2.sum.j = ni12 + ni22 + ni32 + ni42 + ni52
    dlj1.sum.j = exp(dl11) + exp(dl21) + exp(dl31) + exp(dl41) + exp(dl51)
    dlj2.sum.j = exp(dl12) + exp(dl22) + exp(dl32) + exp(dl42) + exp(dl52)
    s11 = alpha1_exp + linear1_exp * dlj1.sum.j
    s12 = alpha1_exp + nij1.sum.j
    s21 = alpha2_exp + linear2_exp * dlj2.sum.j
    s22 = alpha2_exp + nij2.sum.j
    c1 = (s11/(s11+1))^(s12) - (alpha1_exp/(alpha1_exp+1))^(alpha1_exp)
    c2 = (s21/(s21+1))^(s22) - (alpha2_exp/(alpha2_exp+1))^(alpha2_exp)
    denominator = 1 + omega * c1 * c2

    return (sum(digamma(s12) - digamma(alpha1_exp) + alpha1_exp + alpha1 * alpha1_exp - alpha1_exp * log(s11) - alpha1_exp * s12 / s11 + 
                   omega * c2 * ( c1 * (alpha1_exp * log(s11/(s11+1)) + s12 * (alpha1_exp/s11 - alpha1_exp/(s11+1))) - (alpha1_exp/(alpha1_exp))^(alpha1_exp) *
                   alpha1_exp * (1 - alpha1_exp/(alpha1_exp+1) + log(alpha1_exp/(alpha1_exp+1))) ) / denominator))
}

p_beta10 = function(par, data){
    alpha1 = par[1]
    beta10 = par[2]
    beta11 = par[3]
    beta12 = par[4]
    dl11 = par[5]
    dl21 = par[6]
    dl31 = par[7]
    dl41 = par[8]
    dl51 = par[9]
    alpha2 = par[10]
    beta20 = par[11]
    beta21 = par[12]
    beta22 = par[13]
    dl12 = par[14]
    dl22 = par[15]
    dl32 = par[16]
    dl42 = par[17]
    dl52 = par[18]
    omega = par[19]

    ni11 = data[, 1]
    ni21 = data[, 2]
    ni31 = data[, 3]
    ni41 = data[, 4]
    ni51 = data[, 5]
    ni12 = data[, 6]
    ni22 = data[, 7]
    ni32 = data[, 8]
    ni42 = data[, 9]
    ni52 = data[, 10]
    z1 = data[, 11]
    z2 = data[, 12]

    alpha1_exp = exp(alpha1)
    alpha2_exp = exp(alpha2)
    linear1 = beta10 + beta11 * z1 + beta12 * z2
    linear2 = beta20 + beta21 * z1 + beta22 * z2
    linear1_exp = exp(linear1)
    linear2_exp = exp(linear2)
    nij1.sum.j = ni11 + ni21 + ni31 + ni41 + ni51
    nij2.sum.j = ni12 + ni22 + ni32 + ni42 + ni52
    dlj1.sum.j = exp(dl11) + exp(dl21) + exp(dl31) + exp(dl41) + exp(dl51)
    dlj2.sum.j = exp(dl12) + exp(dl22) + exp(dl32) + exp(dl42) + exp(dl52)
    s11 = alpha1_exp + linear1_exp * dlj1.sum.j
    s12 = alpha1_exp + nij1.sum.j
    s21 = alpha2_exp + linear2_exp * dlj2.sum.j
    s22 = alpha2_exp + nij2.sum.j
    c1 = (s11/(s11+1))^(s12) - (alpha1_exp/(alpha1_exp+1))^(alpha1_exp)
    c2 = (s21/(s21+1))^(s22) - (alpha2_exp/(alpha2_exp+1))^(alpha2_exp)
    denominator = 1 + omega * c1 * c2

    return (sum(nij1.sum.j - s12 * linear1_exp * dlj1.sum.j / s11 + omega * c2 * (s11/(s11+1))^(s12) * s12 * (linear1_exp*dlj1.sum.j/s11 - linear1_exp*dlj1.sum.j/(s11+1)) / denominator))
}

p_beta11 = function(par, data){
    alpha1 = par[1]
    beta10 = par[2]
    beta11 = par[3]
    beta12 = par[4]
    dl11 = par[5]
    dl21 = par[6]
    dl31 = par[7]
    dl41 = par[8]
    dl51 = par[9]
    alpha2 = par[10]
    beta20 = par[11]
    beta21 = par[12]
    beta22 = par[13]
    dl12 = par[14]
    dl22 = par[15]
    dl32 = par[16]
    dl42 = par[17]
    dl52 = par[18]
    omega = par[19]

    ni11 = data[, 1]
    ni21 = data[, 2]
    ni31 = data[, 3]
    ni41 = data[, 4]
    ni51 = data[, 5]
    ni12 = data[, 6]
    ni22 = data[, 7]
    ni32 = data[, 8]
    ni42 = data[, 9]
    ni52 = data[, 10]
    z1 = data[, 11]
    z2 = data[, 12]

    alpha1_exp = exp(alpha1)
    alpha2_exp = exp(alpha2)
    linear1 = beta10 + beta11 * z1 + beta12 * z2
    linear2 = beta20 + beta21 * z1 + beta22 * z2
    linear1_exp = exp(linear1)
    linear2_exp = exp(linear2)
    nij1.sum.j = ni11 + ni21 + ni31 + ni41 + ni51
    nij2.sum.j = ni12 + ni22 + ni32 + ni42 + ni52
    dlj1.sum.j = exp(dl11) + exp(dl21) + exp(dl31) + exp(dl41) + exp(dl51)
    dlj2.sum.j = exp(dl12) + exp(dl22) + exp(dl32) + exp(dl42) + exp(dl52)
    s11 = alpha1_exp + linear1_exp * dlj1.sum.j
    s12 = alpha1_exp + nij1.sum.j
    s21 = alpha2_exp + linear2_exp * dlj2.sum.j
    s22 = alpha2_exp + nij2.sum.j
    c1 = (s11/(s11+1))^(s12) - (alpha1_exp/(alpha1_exp+1))^(alpha1_exp)
    c2 = (s21/(s21+1))^(s22) - (alpha2_exp/(alpha2_exp+1))^(alpha2_exp)
    denominator = 1 + omega * c1 * c2

    return (sum(z1 * nij1.sum.j - s12 * z1 * linear1_exp * dlj1.sum.j / s11 + omega * c2 * (s11/(s11+1))^(s12) * s12 * (z1*linear1_exp*dlj1.sum.j/s11 - z1*linear1_exp*dlj1.sum.j/(s11+1)) / denominator))
}

p_beta12 = function(par, data){
    alpha1 = par[1]
    beta10 = par[2]
    beta11 = par[3]
    beta12 = par[4]
    dl11 = par[5]
    dl21 = par[6]
    dl31 = par[7]
    dl41 = par[8]
    dl51 = par[9]
    alpha2 = par[10]
    beta20 = par[11]
    beta21 = par[12]
    beta22 = par[13]
    dl12 = par[14]
    dl22 = par[15]
    dl32 = par[16]
    dl42 = par[17]
    dl52 = par[18]
    omega = par[19]

    ni11 = data[, 1]
    ni21 = data[, 2]
    ni31 = data[, 3]
    ni41 = data[, 4]
    ni51 = data[, 5]
    ni12 = data[, 6]
    ni22 = data[, 7]
    ni32 = data[, 8]
    ni42 = data[, 9]
    ni52 = data[, 10]
    z1 = data[, 11]
    z2 = data[, 12]

    alpha1_exp = exp(alpha1)
    alpha2_exp = exp(alpha2)
    linear1 = beta10 + beta11 * z1 + beta12 * z2
    linear2 = beta20 + beta21 * z1 + beta22 * z2
    linear1_exp = exp(linear1)
    linear2_exp = exp(linear2)
    nij1.sum.j = ni11 + ni21 + ni31 + ni41 + ni51
    nij2.sum.j = ni12 + ni22 + ni32 + ni42 + ni52
    dlj1.sum.j = exp(dl11) + exp(dl21) + exp(dl31) + exp(dl41) + exp(dl51)
    dlj2.sum.j = exp(dl12) + exp(dl22) + exp(dl32) + exp(dl42) + exp(dl52)
    s11 = alpha1_exp + linear1_exp * dlj1.sum.j
    s12 = alpha1_exp + nij1.sum.j
    s21 = alpha2_exp + linear2_exp * dlj2.sum.j
    s22 = alpha2_exp + nij2.sum.j
    c1 = (s11/(s11+1))^(s12) - (alpha1_exp/(alpha1_exp+1))^(alpha1_exp)
    c2 = (s21/(s21+1))^(s22) - (alpha2_exp/(alpha2_exp+1))^(alpha2_exp)
    denominator = 1 + omega * c1 * c2

    return (sum(z2 * nij1.sum.j - s12 * z2 * linear1_exp * dlj1.sum.j / s11 + omega * c2 * (s11/(s11+1))^(s12) * s12 * (z2*linear1_exp*dlj1.sum.j/s11 - z2*linear1_exp*dlj1.sum.j/(s11+1)) / denominator))
}

p_dl11 = function(par, data){
    alpha1 = par[1]
    beta10 = par[2]
    beta11 = par[3]
    beta12 = par[4]
    dl11 = par[5]
    dl21 = par[6]
    dl31 = par[7]
    dl41 = par[8]
    dl51 = par[9]
    alpha2 = par[10]
    beta20 = par[11]
    beta21 = par[12]
    beta22 = par[13]
    dl12 = par[14]
    dl22 = par[15]
    dl32 = par[16]
    dl42 = par[17]
    dl52 = par[18]
    omega = par[19]

    ni11 = data[, 1]
    ni21 = data[, 2]
    ni31 = data[, 3]
    ni41 = data[, 4]
    ni51 = data[, 5]
    ni12 = data[, 6]
    ni22 = data[, 7]
    ni32 = data[, 8]
    ni42 = data[, 9]
    ni52 = data[, 10]
    z1 = data[, 11]
    z2 = data[, 12]

    alpha1_exp = exp(alpha1)
    alpha2_exp = exp(alpha2)
    linear1 = beta10 + beta11 * z1 + beta12 * z2
    linear2 = beta20 + beta21 * z1 + beta22 * z2
    linear1_exp = exp(linear1)
    linear2_exp = exp(linear2)
    nij1.sum.j = ni11 + ni21 + ni31 + ni41 + ni51
    nij2.sum.j = ni12 + ni22 + ni32 + ni42 + ni52
    dlj1.sum.j = exp(dl11) + exp(dl21) + exp(dl31) + exp(dl41) + exp(dl51)
    dlj2.sum.j = exp(dl12) + exp(dl22) + exp(dl32) + exp(dl42) + exp(dl52)
    s11 = alpha1_exp + linear1_exp * dlj1.sum.j
    s12 = alpha1_exp + nij1.sum.j
    s21 = alpha2_exp + linear2_exp * dlj2.sum.j
    s22 = alpha2_exp + nij2.sum.j
    c1 = (s11/(s11+1))^(s12) - (alpha1_exp/(alpha1_exp+1))^(alpha1_exp)
    c2 = (s21/(s21+1))^(s22) - (alpha2_exp/(alpha2_exp+1))^(alpha2_exp)
    denominator = 1 + omega * c1 * c2
    return (sum(ni11 - s12 * linear1_exp * dl11/s11 + omega * c2 * (s11/(s11+1))^(s12) * s12 * (linear1_exp*dl11/s11 - linear1_exp*dl11/(s11+1)) / denominator))
}

p_dl21 = function(par, data){
    alpha1 = par[1]
    beta10 = par[2]
    beta11 = par[3]
    beta12 = par[4]
    dl11 = par[5]
    dl21 = par[6]
    dl31 = par[7]
    dl41 = par[8]
    dl51 = par[9]
    alpha2 = par[10]
    beta20 = par[11]
    beta21 = par[12]
    beta22 = par[13]
    dl12 = par[14]
    dl22 = par[15]
    dl32 = par[16]
    dl42 = par[17]
    dl52 = par[18]
    omega = par[19]

    ni11 = data[, 1]
    ni21 = data[, 2]
    ni31 = data[, 3]
    ni41 = data[, 4]
    ni51 = data[, 5]
    ni12 = data[, 6]
    ni22 = data[, 7]
    ni32 = data[, 8]
    ni42 = data[, 9]
    ni52 = data[, 10]
    z1 = data[, 11]
    z2 = data[, 12]

    alpha1_exp = exp(alpha1)
    alpha2_exp = exp(alpha2)
    linear1 = beta10 + beta11 * z1 + beta12 * z2
    linear2 = beta20 + beta21 * z1 + beta22 * z2
    linear1_exp = exp(linear1)
    linear2_exp = exp(linear2)
    nij1.sum.j = ni11 + ni21 + ni31 + ni41 + ni51
    nij2.sum.j = ni12 + ni22 + ni32 + ni42 + ni52
    dlj1.sum.j = exp(dl11) + exp(dl21) + exp(dl31) + exp(dl41) + exp(dl51)
    dlj2.sum.j = exp(dl12) + exp(dl22) + exp(dl32) + exp(dl42) + exp(dl52)
    s11 = alpha1_exp + linear1_exp * dlj1.sum.j
    s12 = alpha1_exp + nij1.sum.j
    s21 = alpha2_exp + linear2_exp * dlj2.sum.j
    s22 = alpha2_exp + nij2.sum.j
    c1 = (s11/(s11+1))^(s12) - (alpha1_exp/(alpha1_exp+1))^(alpha1_exp)
    c2 = (s21/(s21+1))^(s22) - (alpha2_exp/(alpha2_exp+1))^(alpha2_exp)
    denominator = 1 + omega * c1 * c2
    return (sum(ni21 - s12 * linear1_exp * dl21/s11 + omega * c2 * (s11/(s11+1))^(s12) * s12 * (linear1_exp*dl21/s11 - linear1_exp*dl21/(s11+1)) / denominator))
}

p_dl31 = function(par, data){
    alpha1 = par[1]
    beta10 = par[2]
    beta11 = par[3]
    beta12 = par[4]
    dl11 = par[5]
    dl21 = par[6]
    dl31 = par[7]
    dl41 = par[8]
    dl51 = par[9]
    alpha2 = par[10]
    beta20 = par[11]
    beta21 = par[12]
    beta22 = par[13]
    dl12 = par[14]
    dl22 = par[15]
    dl32 = par[16]
    dl42 = par[17]
    dl52 = par[18]
    omega = par[19]

    ni11 = data[, 1]
    ni21 = data[, 2]
    ni31 = data[, 3]
    ni41 = data[, 4]
    ni51 = data[, 5]
    ni12 = data[, 6]
    ni22 = data[, 7]
    ni32 = data[, 8]
    ni42 = data[, 9]
    ni52 = data[, 10]
    z1 = data[, 11]
    z2 = data[, 12]

    alpha1_exp = exp(alpha1)
    alpha2_exp = exp(alpha2)
    linear1 = beta10 + beta11 * z1 + beta12 * z2
    linear2 = beta20 + beta21 * z1 + beta22 * z2
    linear1_exp = exp(linear1)
    linear2_exp = exp(linear2)
    nij1.sum.j = ni11 + ni21 + ni31 + ni41 + ni51
    nij2.sum.j = ni12 + ni22 + ni32 + ni42 + ni52
    dlj1.sum.j = exp(dl11) + exp(dl21) + exp(dl31) + exp(dl41) + exp(dl51)
    dlj2.sum.j = exp(dl12) + exp(dl22) + exp(dl32) + exp(dl42) + exp(dl52)
    s11 = alpha1_exp + linear1_exp * dlj1.sum.j
    s12 = alpha1_exp + nij1.sum.j
    s21 = alpha2_exp + linear2_exp * dlj2.sum.j
    s22 = alpha2_exp + nij2.sum.j
    c1 = (s11/(s11+1))^(s12) - (alpha1_exp/(alpha1_exp+1))^(alpha1_exp)
    c2 = (s21/(s21+1))^(s22) - (alpha2_exp/(alpha2_exp+1))^(alpha2_exp)
    denominator = 1 + omega * c1 * c2
    return (sum(ni31 - s12 * linear1_exp * dl31/s11 + omega * c2 * (s11/(s11+1))^(s12) * s12 * (linear1_exp*dl31/s11 - linear1_exp*dl31/(s11+1)) / denominator))
}

p_dl41 = function(par, data){
    alpha1 = par[1]
    beta10 = par[2]
    beta11 = par[3]
    beta12 = par[4]
    dl11 = par[5]
    dl21 = par[6]
    dl31 = par[7]
    dl41 = par[8]
    dl51 = par[9]
    alpha2 = par[10]
    beta20 = par[11]
    beta21 = par[12]
    beta22 = par[13]
    dl12 = par[14]
    dl22 = par[15]
    dl32 = par[16]
    dl42 = par[17]
    dl52 = par[18]
    omega = par[19]

    ni11 = data[, 1]
    ni21 = data[, 2]
    ni31 = data[, 3]
    ni41 = data[, 4]
    ni51 = data[, 5]
    ni12 = data[, 6]
    ni22 = data[, 7]
    ni32 = data[, 8]
    ni42 = data[, 9]
    ni52 = data[, 10]
    z1 = data[, 11]
    z2 = data[, 12]

    alpha1_exp = exp(alpha1)
    alpha2_exp = exp(alpha2)
    linear1 = beta10 + beta11 * z1 + beta12 * z2
    linear2 = beta20 + beta21 * z1 + beta22 * z2
    linear1_exp = exp(linear1)
    linear2_exp = exp(linear2)
    nij1.sum.j = ni11 + ni21 + ni31 + ni41 + ni51
    nij2.sum.j = ni12 + ni22 + ni32 + ni42 + ni52
    dlj1.sum.j = exp(dl11) + exp(dl21) + exp(dl31) + exp(dl41) + exp(dl51)
    dlj2.sum.j = exp(dl12) + exp(dl22) + exp(dl32) + exp(dl42) + exp(dl52)
    s11 = alpha1_exp + linear1_exp * dlj1.sum.j
    s12 = alpha1_exp + nij1.sum.j
    s21 = alpha2_exp + linear2_exp * dlj2.sum.j
    s22 = alpha2_exp + nij2.sum.j
    c1 = (s11/(s11+1))^(s12) - (alpha1_exp/(alpha1_exp+1))^(alpha1_exp)
    c2 = (s21/(s21+1))^(s22) - (alpha2_exp/(alpha2_exp+1))^(alpha2_exp)
    denominator = 1 + omega * c1 * c2
    return (sum(ni41 - s12 * linear1_exp * dl41/s11 + omega * c2 * (s11/(s11+1))^(s12) * s12 * (linear1_exp*dl41/s11 - linear1_exp*dl41/(s11+1)) / denominator))
}

p_dl51 = function(par, data){
    alpha1 = par[1]
    beta10 = par[2]
    beta11 = par[3]
    beta12 = par[4]
    dl11 = par[5]
    dl21 = par[6]
    dl31 = par[7]
    dl41 = par[8]
    dl51 = par[9]
    alpha2 = par[10]
    beta20 = par[11]
    beta21 = par[12]
    beta22 = par[13]
    dl12 = par[14]
    dl22 = par[15]
    dl32 = par[16]
    dl42 = par[17]
    dl52 = par[18]
    omega = par[19]

    ni11 = data[, 1]
    ni21 = data[, 2]
    ni31 = data[, 3]
    ni41 = data[, 4]
    ni51 = data[, 5]
    ni12 = data[, 6]
    ni22 = data[, 7]
    ni32 = data[, 8]
    ni42 = data[, 9]
    ni52 = data[, 10]
    z1 = data[, 11]
    z2 = data[, 12]

    alpha1_exp = exp(alpha1)
    alpha2_exp = exp(alpha2)
    linear1 = beta10 + beta11 * z1 + beta12 * z2
    linear2 = beta20 + beta21 * z1 + beta22 * z2
    linear1_exp = exp(linear1)
    linear2_exp = exp(linear2)
    nij1.sum.j = ni11 + ni21 + ni31 + ni41 + ni51
    nij2.sum.j = ni12 + ni22 + ni32 + ni42 + ni52
    dlj1.sum.j = exp(dl11) + exp(dl21) + exp(dl31) + exp(dl41) + exp(dl51)
    dlj2.sum.j = exp(dl12) + exp(dl22) + exp(dl32) + exp(dl42) + exp(dl52)
    s11 = alpha1_exp + linear1_exp * dlj1.sum.j
    s12 = alpha1_exp + nij1.sum.j
    s21 = alpha2_exp + linear2_exp * dlj2.sum.j
    s22 = alpha2_exp + nij2.sum.j
    c1 = (s11/(s11+1))^(s12) - (alpha1_exp/(alpha1_exp+1))^(alpha1_exp)
    c2 = (s21/(s21+1))^(s22) - (alpha2_exp/(alpha2_exp+1))^(alpha2_exp)
    denominator = 1 + omega * c1 * c2
    return (sum(ni51 - s12 * linear1_exp * dl51/s11 + omega * c2 * (s11/(s11+1))^(s12) * s12 * (linear1_exp*dl51/s11 - linear1_exp*dl51/(s11+1)) / denominator))
}

In [97]:
p_alpha2 = function(par, data){
    alpha1 = par[1]
    beta10 = par[2]
    beta11 = par[3]
    beta12 = par[4]
    dl11 = par[5]
    dl21 = par[6]
    dl31 = par[7]
    dl41 = par[8]
    dl51 = par[9]
    alpha2 = par[10]
    beta20 = par[11]
    beta21 = par[12]
    beta22 = par[13]
    dl12 = par[14]
    dl22 = par[15]
    dl32 = par[16]
    dl42 = par[17]
    dl52 = par[18]
    omega = par[19]

    ni11 = data[, 1]
    ni21 = data[, 2]
    ni31 = data[, 3]
    ni41 = data[, 4]
    ni51 = data[, 5]
    ni12 = data[, 6]
    ni22 = data[, 7]
    ni32 = data[, 8]
    ni42 = data[, 9]
    ni52 = data[, 10]
    z1 = data[, 11]
    z2 = data[, 12]

    alpha1_exp = exp(alpha1)
    alpha2_exp = exp(alpha2)
    linear1 = beta10 + beta11 * z1 + beta12 * z2
    linear2 = beta20 + beta21 * z1 + beta22 * z2
    linear1_exp = exp(linear1)
    linear2_exp = exp(linear2)
    nij1.sum.j = ni11 + ni21 + ni31 + ni41 + ni51
    nij2.sum.j = ni12 + ni22 + ni32 + ni42 + ni52
    dlj1.sum.j = exp(dl11) + exp(dl21) + exp(dl31) + exp(dl41) + exp(dl51)
    dlj2.sum.j = exp(dl12) + exp(dl22) + exp(dl32) + exp(dl42) + exp(dl52)
    s11 = alpha1_exp + linear1_exp * dlj1.sum.j
    s12 = alpha1_exp + nij1.sum.j
    s21 = alpha2_exp + linear2_exp * dlj2.sum.j
    s22 = alpha2_exp + nij2.sum.j
    c1 = (s11/(s11+1))^(s12) - (alpha1_exp/(alpha1_exp+1))^(alpha1_exp)
    c2 = (s21/(s21+1))^(s22) - (alpha2_exp/(alpha2_exp+1))^(alpha2_exp)
    denominator = 1 + omega * c1 * c2
    return (sum(digamma(s22) - digamma(alpha2_exp) + alpha2_exp + alpha2 * alpha2_exp - alpha2_exp * log(s21) - alpha2_exp * s22 / s21 + 
                   omega * c1 * ( c2 * (alpha2_exp * log(s21/(s21+1)) + s22 * (alpha2_exp/s21 - alpha2_exp/(s21+1))) - (alpha2_exp/(alpha2_exp))^(alpha2_exp) *
                   alpha2_exp * (1 - alpha2_exp/(alpha2_exp+1) + log(alpha2_exp/(alpha2_exp+1))) ) / denominator))
}

p_beta20 = function(par, data){
    alpha1 = par[1]
    beta10 = par[2]
    beta11 = par[3]
    beta12 = par[4]
    dl11 = par[5]
    dl21 = par[6]
    dl31 = par[7]
    dl41 = par[8]
    dl51 = par[9]
    alpha2 = par[10]
    beta20 = par[11]
    beta21 = par[12]
    beta22 = par[13]
    dl12 = par[14]
    dl22 = par[15]
    dl32 = par[16]
    dl42 = par[17]
    dl52 = par[18]
    omega = par[19]

    ni11 = data[, 1]
    ni21 = data[, 2]
    ni31 = data[, 3]
    ni41 = data[, 4]
    ni51 = data[, 5]
    ni12 = data[, 6]
    ni22 = data[, 7]
    ni32 = data[, 8]
    ni42 = data[, 9]
    ni52 = data[, 10]
    z1 = data[, 11]
    z2 = data[, 12]

    alpha1_exp = exp(alpha1)
    alpha2_exp = exp(alpha2)
    linear1 = beta10 + beta11 * z1 + beta12 * z2
    linear2 = beta20 + beta21 * z1 + beta22 * z2
    linear1_exp = exp(linear1)
    linear2_exp = exp(linear2)
    nij1.sum.j = ni11 + ni21 + ni31 + ni41 + ni51
    nij2.sum.j = ni12 + ni22 + ni32 + ni42 + ni52
    dlj1.sum.j = exp(dl11) + exp(dl21) + exp(dl31) + exp(dl41) + exp(dl51)
    dlj2.sum.j = exp(dl12) + exp(dl22) + exp(dl32) + exp(dl42) + exp(dl52)
    s11 = alpha1_exp + linear1_exp * dlj1.sum.j
    s12 = alpha1_exp + nij1.sum.j
    s21 = alpha2_exp + linear2_exp * dlj2.sum.j
    s22 = alpha2_exp + nij2.sum.j
    c1 = (s11/(s11+1))^(s12) - (alpha1_exp/(alpha1_exp+1))^(alpha1_exp)
    c2 = (s21/(s21+1))^(s22) - (alpha2_exp/(alpha2_exp+1))^(alpha2_exp)
    denominator = 1 + omega * c1 * c2
    return (sum(nij2.sum.j - s22 * linear2_exp * dlj2.sum.j / s21 + omega * c1 * (s21/(s21+1))^(s22) * s22 * (linear2_exp*dlj2.sum.j/s21 - linear2_exp*dlj2.sum.j/(s21+1)) / denominator))
}

p_beta21 = function(par, data){
    alpha1 = par[1]
    beta10 = par[2]
    beta11 = par[3]
    beta12 = par[4]
    dl11 = par[5]
    dl21 = par[6]
    dl31 = par[7]
    dl41 = par[8]
    dl51 = par[9]
    alpha2 = par[10]
    beta20 = par[11]
    beta21 = par[12]
    beta22 = par[13]
    dl12 = par[14]
    dl22 = par[15]
    dl32 = par[16]
    dl42 = par[17]
    dl52 = par[18]
    omega = par[19]

    ni11 = data[, 1]
    ni21 = data[, 2]
    ni31 = data[, 3]
    ni41 = data[, 4]
    ni51 = data[, 5]
    ni12 = data[, 6]
    ni22 = data[, 7]
    ni32 = data[, 8]
    ni42 = data[, 9]
    ni52 = data[, 10]
    z1 = data[, 11]
    z2 = data[, 12]

    alpha1_exp = exp(alpha1)
    alpha2_exp = exp(alpha2)
    linear1 = beta10 + beta11 * z1 + beta12 * z2
    linear2 = beta20 + beta21 * z1 + beta22 * z2
    linear1_exp = exp(linear1)
    linear2_exp = exp(linear2)
    nij1.sum.j = ni11 + ni21 + ni31 + ni41 + ni51
    nij2.sum.j = ni12 + ni22 + ni32 + ni42 + ni52
    dlj1.sum.j = exp(dl11) + exp(dl21) + exp(dl31) + exp(dl41) + exp(dl51)
    dlj2.sum.j = exp(dl12) + exp(dl22) + exp(dl32) + exp(dl42) + exp(dl52)
    s11 = alpha1_exp + linear1_exp * dlj1.sum.j
    s12 = alpha1_exp + nij1.sum.j
    s21 = alpha2_exp + linear2_exp * dlj2.sum.j
    s22 = alpha2_exp + nij2.sum.j
    c1 = (s11/(s11+1))^(s12) - (alpha1_exp/(alpha1_exp+1))^(alpha1_exp)
    c2 = (s21/(s21+1))^(s22) - (alpha2_exp/(alpha2_exp+1))^(alpha2_exp)
    denominator = 1 + omega * c1 * c2
    return (sum(z1 * nij2.sum.j - s22 * z1 * linear2_exp * dlj2.sum.j / s21 + omega * c1 * (s11/(s21+1))^(s22) * s22 * (z1*linear2_exp*dlj2.sum.j/s21 - z1*linear2_exp*dlj1.sum.j/(s21+1)) / denominator))
}

p_beta22 = function(par, data){
    alpha1 = par[1]
    beta10 = par[2]
    beta11 = par[3]
    beta12 = par[4]
    dl11 = par[5]
    dl21 = par[6]
    dl31 = par[7]
    dl41 = par[8]
    dl51 = par[9]
    alpha2 = par[10]
    beta20 = par[11]
    beta21 = par[12]
    beta22 = par[13]
    dl12 = par[14]
    dl22 = par[15]
    dl32 = par[16]
    dl42 = par[17]
    dl52 = par[18]
    omega = par[19]

    ni11 = data[, 1]
    ni21 = data[, 2]
    ni31 = data[, 3]
    ni41 = data[, 4]
    ni51 = data[, 5]
    ni12 = data[, 6]
    ni22 = data[, 7]
    ni32 = data[, 8]
    ni42 = data[, 9]
    ni52 = data[, 10]
    z1 = data[, 11]
    z2 = data[, 12]

    alpha1_exp = exp(alpha1)
    alpha2_exp = exp(alpha2)
    linear1 = beta10 + beta11 * z1 + beta12 * z2
    linear2 = beta20 + beta21 * z1 + beta22 * z2
    linear1_exp = exp(linear1)
    linear2_exp = exp(linear2)
    nij1.sum.j = ni11 + ni21 + ni31 + ni41 + ni51
    nij2.sum.j = ni12 + ni22 + ni32 + ni42 + ni52
    dlj1.sum.j = exp(dl11) + exp(dl21) + exp(dl31) + exp(dl41) + exp(dl51)
    dlj2.sum.j = exp(dl12) + exp(dl22) + exp(dl32) + exp(dl42) + exp(dl52)
    s11 = alpha1_exp + linear1_exp * dlj1.sum.j
    s12 = alpha1_exp + nij1.sum.j
    s21 = alpha2_exp + linear2_exp * dlj2.sum.j
    s22 = alpha2_exp + nij2.sum.j
    c1 = (s11/(s11+1))^(s12) - (alpha1_exp/(alpha1_exp+1))^(alpha1_exp)
    c2 = (s21/(s21+1))^(s22) - (alpha2_exp/(alpha2_exp+1))^(alpha2_exp)
    denominator = 1 + omega * c1 * c2
    return (sum(z2 * nij2.sum.j - s22 * z2 * linear2_exp * dlj2.sum.j / s21 + omega * c1 * (s11/(s21+1))^(s22) * s22 * (z2*linear2_exp*dlj2.sum.j/s21 - z2*linear2_exp*dlj1.sum.j/(s21+1)) / denominator))
}

p_dl12 = function(par, data){
    alpha1 = par[1]
    beta10 = par[2]
    beta11 = par[3]
    beta12 = par[4]
    dl11 = par[5]
    dl21 = par[6]
    dl31 = par[7]
    dl41 = par[8]
    dl51 = par[9]
    alpha2 = par[10]
    beta20 = par[11]
    beta21 = par[12]
    beta22 = par[13]
    dl12 = par[14]
    dl22 = par[15]
    dl32 = par[16]
    dl42 = par[17]
    dl52 = par[18]
    omega = par[19]

    ni11 = data[, 1]
    ni21 = data[, 2]
    ni31 = data[, 3]
    ni41 = data[, 4]
    ni51 = data[, 5]
    ni12 = data[, 6]
    ni22 = data[, 7]
    ni32 = data[, 8]
    ni42 = data[, 9]
    ni52 = data[, 10]
    z1 = data[, 11]
    z2 = data[, 12]

    alpha1_exp = exp(alpha1)
    alpha2_exp = exp(alpha2)
    linear1 = beta10 + beta11 * z1 + beta12 * z2
    linear2 = beta20 + beta21 * z1 + beta22 * z2
    linear1_exp = exp(linear1)
    linear2_exp = exp(linear2)
    nij1.sum.j = ni11 + ni21 + ni31 + ni41 + ni51
    nij2.sum.j = ni12 + ni22 + ni32 + ni42 + ni52
    dlj1.sum.j = exp(dl11) + exp(dl21) + exp(dl31) + exp(dl41) + exp(dl51)
    dlj2.sum.j = exp(dl12) + exp(dl22) + exp(dl32) + exp(dl42) + exp(dl52)
    s11 = alpha1_exp + linear1_exp * dlj1.sum.j
    s12 = alpha1_exp + nij1.sum.j
    s21 = alpha2_exp + linear2_exp * dlj2.sum.j
    s22 = alpha2_exp + nij2.sum.j
    c1 = (s11/(s11+1))^(s12) - (alpha1_exp/(alpha1_exp+1))^(alpha1_exp)
    c2 = (s21/(s21+1))^(s22) - (alpha2_exp/(alpha2_exp+1))^(alpha2_exp)
    denominator = 1 + omega * c1 * c2
    return (sum(ni12 - s22 * linear2_exp * dl12/s21 + omega * c1 * (s21/(s21+1))^(s22) * s22 * (linear2_exp*dl12/s21 - linear2_exp*dl12/(s21+1)) / denominator))
}

p_dl22 = function(par, data){
    alpha1 = par[1]
    beta10 = par[2]
    beta11 = par[3]
    beta12 = par[4]
    dl11 = par[5]
    dl21 = par[6]
    dl31 = par[7]
    dl41 = par[8]
    dl51 = par[9]
    alpha2 = par[10]
    beta20 = par[11]
    beta21 = par[12]
    beta22 = par[13]
    dl12 = par[14]
    dl22 = par[15]
    dl32 = par[16]
    dl42 = par[17]
    dl52 = par[18]
    omega = par[19]

    ni11 = data[, 1]
    ni21 = data[, 2]
    ni31 = data[, 3]
    ni41 = data[, 4]
    ni51 = data[, 5]
    ni12 = data[, 6]
    ni22 = data[, 7]
    ni32 = data[, 8]
    ni42 = data[, 9]
    ni52 = data[, 10]
    z1 = data[, 11]
    z2 = data[, 12]

    alpha1_exp = exp(alpha1)
    alpha2_exp = exp(alpha2)
    linear1 = beta10 + beta11 * z1 + beta12 * z2
    linear2 = beta20 + beta21 * z1 + beta22 * z2
    linear1_exp = exp(linear1)
    linear2_exp = exp(linear2)
    nij1.sum.j = ni11 + ni21 + ni31 + ni41 + ni51
    nij2.sum.j = ni12 + ni22 + ni32 + ni42 + ni52
    dlj1.sum.j = exp(dl11) + exp(dl21) + exp(dl31) + exp(dl41) + exp(dl51)
    dlj2.sum.j = exp(dl12) + exp(dl22) + exp(dl32) + exp(dl42) + exp(dl52)
    s11 = alpha1_exp + linear1_exp * dlj1.sum.j
    s12 = alpha1_exp + nij1.sum.j
    s21 = alpha2_exp + linear2_exp * dlj2.sum.j
    s22 = alpha2_exp + nij2.sum.j
    c1 = (s11/(s11+1))^(s12) - (alpha1_exp/(alpha1_exp+1))^(alpha1_exp)
    c2 = (s21/(s21+1))^(s22) - (alpha2_exp/(alpha2_exp+1))^(alpha2_exp)
    denominator = 1 + omega * c1 * c2
    return (sum(ni22 - s22 * linear2_exp * dl22/s21 + omega * c1 * (s21/(s21+1))^(s22) * s22 * (linear2_exp*dl22/s21 - linear2_exp*dl22/(s21+1)) / denominator))
}

p_dl32 = function(par, data){
    alpha1 = par[1]
    beta10 = par[2]
    beta11 = par[3]
    beta12 = par[4]
    dl11 = par[5]
    dl21 = par[6]
    dl31 = par[7]
    dl41 = par[8]
    dl51 = par[9]
    alpha2 = par[10]
    beta20 = par[11]
    beta21 = par[12]
    beta22 = par[13]
    dl12 = par[14]
    dl22 = par[15]
    dl32 = par[16]
    dl42 = par[17]
    dl52 = par[18]
    omega = par[19]

    ni11 = data[, 1]
    ni21 = data[, 2]
    ni31 = data[, 3]
    ni41 = data[, 4]
    ni51 = data[, 5]
    ni12 = data[, 6]
    ni22 = data[, 7]
    ni32 = data[, 8]
    ni42 = data[, 9]
    ni52 = data[, 10]
    z1 = data[, 11]
    z2 = data[, 12]

    alpha1_exp = exp(alpha1)
    alpha2_exp = exp(alpha2)
    linear1 = beta10 + beta11 * z1 + beta12 * z2
    linear2 = beta20 + beta21 * z1 + beta22 * z2
    linear1_exp = exp(linear1)
    linear2_exp = exp(linear2)
    nij1.sum.j = ni11 + ni21 + ni31 + ni41 + ni51
    nij2.sum.j = ni12 + ni22 + ni32 + ni42 + ni52
    dlj1.sum.j = exp(dl11) + exp(dl21) + exp(dl31) + exp(dl41) + exp(dl51)
    dlj2.sum.j = exp(dl12) + exp(dl22) + exp(dl32) + exp(dl42) + exp(dl52)
    s11 = alpha1_exp + linear1_exp * dlj1.sum.j
    s12 = alpha1_exp + nij1.sum.j
    s21 = alpha2_exp + linear2_exp * dlj2.sum.j
    s22 = alpha2_exp + nij2.sum.j
    c1 = (s11/(s11+1))^(s12) - (alpha1_exp/(alpha1_exp+1))^(alpha1_exp)
    c2 = (s21/(s21+1))^(s22) - (alpha2_exp/(alpha2_exp+1))^(alpha2_exp)
    denominator = 1 + omega * c1 * c2
    return (sum(ni32 - s22 * linear2_exp * dl32/s21 + omega * c1 * (s21/(s21+1))^(s22) * s22 * (linear2_exp*dl32/s21 - linear2_exp*dl32/(s21+1)) / denominator))
}

p_dl42 = function(par, data){
    alpha1 = par[1]
    beta10 = par[2]
    beta11 = par[3]
    beta12 = par[4]
    dl11 = par[5]
    dl21 = par[6]
    dl31 = par[7]
    dl41 = par[8]
    dl51 = par[9]
    alpha2 = par[10]
    beta20 = par[11]
    beta21 = par[12]
    beta22 = par[13]
    dl12 = par[14]
    dl22 = par[15]
    dl32 = par[16]
    dl42 = par[17]
    dl52 = par[18]
    omega = par[19]

    ni11 = data[, 1]
    ni21 = data[, 2]
    ni31 = data[, 3]
    ni41 = data[, 4]
    ni51 = data[, 5]
    ni12 = data[, 6]
    ni22 = data[, 7]
    ni32 = data[, 8]
    ni42 = data[, 9]
    ni52 = data[, 10]
    z1 = data[, 11]
    z2 = data[, 12]

    alpha1_exp = exp(alpha1)
    alpha2_exp = exp(alpha2)
    linear1 = beta10 + beta11 * z1 + beta12 * z2
    linear2 = beta20 + beta21 * z1 + beta22 * z2
    linear1_exp = exp(linear1)
    linear2_exp = exp(linear2)
    nij1.sum.j = ni11 + ni21 + ni31 + ni41 + ni51
    nij2.sum.j = ni12 + ni22 + ni32 + ni42 + ni52
    dlj1.sum.j = exp(dl11) + exp(dl21) + exp(dl31) + exp(dl41) + exp(dl51)
    dlj2.sum.j = exp(dl12) + exp(dl22) + exp(dl32) + exp(dl42) + exp(dl52)
    s11 = alpha1_exp + linear1_exp * dlj1.sum.j
    s12 = alpha1_exp + nij1.sum.j
    s21 = alpha2_exp + linear2_exp * dlj2.sum.j
    s22 = alpha2_exp + nij2.sum.j
    c1 = (s11/(s11+1))^(s12) - (alpha1_exp/(alpha1_exp+1))^(alpha1_exp)
    c2 = (s21/(s21+1))^(s22) - (alpha2_exp/(alpha2_exp+1))^(alpha2_exp)
    denominator = 1 + omega * c1 * c2
    return (sum(ni42 - s22 * linear2_exp * dl42/s21 + omega * c1 * (s21/(s21+1))^(s22) * s22 * (linear2_exp*dl42/s21 - linear2_exp*dl42/(s21+1)) / denominator))
}

p_dl52 = function(par, data){
    alpha1 = par[1]
    beta10 = par[2]
    beta11 = par[3]
    beta12 = par[4]
    dl11 = par[5]
    dl21 = par[6]
    dl31 = par[7]
    dl41 = par[8]
    dl51 = par[9]
    alpha2 = par[10]
    beta20 = par[11]
    beta21 = par[12]
    beta22 = par[13]
    dl12 = par[14]
    dl22 = par[15]
    dl32 = par[16]
    dl42 = par[17]
    dl52 = par[18]
    omega = par[19]

    ni11 = data[, 1]
    ni21 = data[, 2]
    ni31 = data[, 3]
    ni41 = data[, 4]
    ni51 = data[, 5]
    ni12 = data[, 6]
    ni22 = data[, 7]
    ni32 = data[, 8]
    ni42 = data[, 9]
    ni52 = data[, 10]
    z1 = data[, 11]
    z2 = data[, 12]

    alpha1_exp = exp(alpha1)
    alpha2_exp = exp(alpha2)
    linear1 = beta10 + beta11 * z1 + beta12 * z2
    linear2 = beta20 + beta21 * z1 + beta22 * z2
    linear1_exp = exp(linear1)
    linear2_exp = exp(linear2)
    nij1.sum.j = ni11 + ni21 + ni31 + ni41 + ni51
    nij2.sum.j = ni12 + ni22 + ni32 + ni42 + ni52
    dlj1.sum.j = exp(dl11) + exp(dl21) + exp(dl31) + exp(dl41) + exp(dl51)
    dlj2.sum.j = exp(dl12) + exp(dl22) + exp(dl32) + exp(dl42) + exp(dl52)
    s11 = alpha1_exp + linear1_exp * dlj1.sum.j
    s12 = alpha1_exp + nij1.sum.j
    s21 = alpha2_exp + linear2_exp * dlj2.sum.j
    s22 = alpha2_exp + nij2.sum.j
    c1 = (s11/(s11+1))^(s12) - (alpha1_exp/(alpha1_exp+1))^(alpha1_exp)
    c2 = (s21/(s21+1))^(s22) - (alpha2_exp/(alpha2_exp+1))^(alpha2_exp)
    denominator = 1 + omega * c1 * c2
    return (sum(ni52 - s22 * linear2_exp * dl52/s21 + omega * c1 * (s21/(s21+1))^(s22) * s22 * (linear2_exp*dl52/s21 - linear2_exp*dl52/(s21+1)) / denominator))
}

p_omega = function(par, data){
    alpha1 = par[1]
    beta10 = par[2]
    beta11 = par[3]
    beta12 = par[4]
    dl11 = par[5]
    dl21 = par[6]
    dl31 = par[7]
    dl41 = par[8]
    dl51 = par[9]
    alpha2 = par[10]
    beta20 = par[11]
    beta21 = par[12]
    beta22 = par[13]
    dl12 = par[14]
    dl22 = par[15]
    dl32 = par[16]
    dl42 = par[17]
    dl52 = par[18]
    omega = par[19]

    ni11 = data[, 1]
    ni21 = data[, 2]
    ni31 = data[, 3]
    ni41 = data[, 4]
    ni51 = data[, 5]
    ni12 = data[, 6]
    ni22 = data[, 7]
    ni32 = data[, 8]
    ni42 = data[, 9]
    ni52 = data[, 10]
    z1 = data[, 11]
    z2 = data[, 12]

    alpha1_exp = exp(alpha1)
    alpha2_exp = exp(alpha2)
    linear1 = beta10 + beta11 * z1 + beta12 * z2
    linear2 = beta20 + beta21 * z1 + beta22 * z2
    linear1_exp = exp(linear1)
    linear2_exp = exp(linear2)
    nij1.sum.j = ni11 + ni21 + ni31 + ni41 + ni51
    nij2.sum.j = ni12 + ni22 + ni32 + ni42 + ni52
    dlj1.sum.j = exp(dl11) + exp(dl21) + exp(dl31) + exp(dl41) + exp(dl51)
    dlj2.sum.j = exp(dl12) + exp(dl22) + exp(dl32) + exp(dl42) + exp(dl52)
    s11 = alpha1_exp + linear1_exp * dlj1.sum.j
    s12 = alpha1_exp + nij1.sum.j
    s21 = alpha2_exp + linear2_exp * dlj2.sum.j
    s22 = alpha2_exp + nij2.sum.j
    c1 = (s11/(s11+1))^(s12) - (alpha1_exp/(alpha1_exp+1))^(alpha1_exp)
    c2 = (s21/(s21+1))^(s22) - (alpha2_exp/(alpha2_exp+1))^(alpha2_exp)
    denominator = 1 + omega * c1 * c2
    return (sum(c1*c2/denominator))
}

In [107]:
dt = generate.data(N = 300, alpha = c(3, 2), beta1 = c(0.5, 0.5, -0.5), beta2 = c(0.5, -0.5, 0.5), omega = 0.5)
 s = c(log(3)+runif(1, -0.05, 0.05), 0.5+runif(1, -0.05, 0.05), 0.5+runif(1, -0.05, 0.05),
      -0.5+runif(1, -0.05, 0.05), log(0.05)+runif(1, -0.01, 0.01), log(0.05)+runif(1, -0.01, 0.01),
      log(0.05)+runif(1, -0.01, 0.01), log(0.05)+runif(1, -0.01, 0.01), log(0.05)+runif(1, -0.01, 0.01),
      log(2)+runif(1, -0.05, 0.05), 0.5+runif(1, -0.05, 0.05), -0.5+runif(1, -0.05, 0.05),
      0.5+runif(1, -0.05, 0.05), log(0.025)+runif(1, -0.01, 0.01), log(0.075)+runif(1, -0.01, 0.01),
      log(0.125)+runif(1, -0.01, 0.01), log(0.175)+runif(1, -0.01, 0.01), log(0.225)+runif(1, -0.01, 0.01),
      0.5+runif(1, -0.01, 0.01))

res = optim(par = s, fn = logL.new, data = dt, method = "BFGS", gr = g, hessian = T)
h = matrix(res$hessian, ncol = 19)
p = res$par
h.inv = solve(-h)

In [108]:
param = c()
sd_list = c()
for (i in 1:100){
    dt = generate.data(N = 300, alpha = c(3, 2), beta1 = c(0.5, 0.5, -0.5), beta2 = c(0.5, -0.5, 0.5), omega = 0.5)
    s = c(log(3)+runif(1, -0.05, 0.05), 0.5+runif(1, -0.05, 0.05), 0.5+runif(1, -0.05, 0.05),
      -0.5+runif(1, -0.05, 0.05), log(0.05)+runif(1, -0.01, 0.01), log(0.05)+runif(1, -0.01, 0.01),
      log(0.05)+runif(1, -0.01, 0.01), log(0.05)+runif(1, -0.01, 0.01), log(0.05)+runif(1, -0.01, 0.01),
      log(2)+runif(1, -0.05, 0.05), 0.5+runif(1, -0.05, 0.05), -0.5+runif(1, -0.05, 0.05),
      0.5+runif(1, -0.05, 0.05), log(0.025)+runif(1, -0.01, 0.01), log(0.075)+runif(1, -0.01, 0.01),
      log(0.125)+runif(1, -0.01, 0.01), log(0.175)+runif(1, -0.01, 0.01), log(0.225)+runif(1, -0.01, 0.01),
      0.5+runif(1, -0.01, 0.01))

    result = optim(par = s, fn = logL.new, data = dt, method = "BFGS", gr = g, hessian = T)
    p = result$par
    p[c(1, 5:9, 10, 14:18)] = exp(p[c(1, 5:9, 10, 14:18)])
    h = result$hessian
    h = solve(-h)
    h = sqrt(diag(h))
    param = rbind(param, p)
    sd_list = rbind(sd_list, h)
}

In [109]:
# mean
print(round(apply(param, 2, mean), 4))
cat("===============================================================================\n")
# bias
print(round(apply(param, 2, mean) - c(3, 0.5, 0.5, -0.5, 0.05, 0.05, 0.05, 0.05, 0.05, 2, 0.5, -0.5, 0.5, 0.025, 0.075, 0.125, 0.175, 0.225, 0.5), 4))
cat("===============================================================================\n")
# standard deviation
print(round(apply(param, 2, sd), 4))

 [1]  3.0180  0.4986  0.5062 -0.5002  0.0500  0.0500  0.0500  0.0500  0.0500
[10]  1.9921  0.4976 -0.5017  0.4989  0.0250  0.0750  0.1250  0.1750  0.2251
[19]  0.4994
 [1]  0.0180 -0.0014  0.0062 -0.0002  0.0000  0.0000  0.0000  0.0000  0.0000
[10] -0.0079 -0.0024 -0.0017 -0.0011  0.0000  0.0000  0.0000  0.0000  0.0001
[19] -0.0006
 [1] 0.0888 0.0291 0.0291 0.0283 0.0003 0.0003 0.0003 0.0003 0.0003 0.0565
[11] 0.0291 0.0303 0.0290 0.0001 0.0005 0.0007 0.0011 0.0013 0.0061


In [110]:
print(apply(sd_list, 2, mean))

 [1] 0.16195442 0.09188266 0.10482318 0.20957241 0.04202682 0.04202347
 [7] 0.04202712 0.04203012 0.04202692 0.06847987 0.06700137 0.06836551
[13] 0.13200880 0.01996530 0.03095622 0.03385050 0.03499901 0.03529733
[19] 8.07697352


Fisher 估計的變異為轉換後的變異，需要用 $\delta$ method 轉換